# International Men’s Cricket – Predicting the Market Value of Individual Cricketers in the India Premier League


### Import relevant modules and import initial datasets

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

### Load data

##March 2023 update
Following the EDA phase in late February, I realised I needed to improve the raw data I was using, mostly in relation to having a sufficient volume, but also in relation to completeness for the players involved. The data I had been using did have enough performance information as it could have for as many of the players as it could have (if that makes sense). So with this version of this Notebook, I've included more and fuller data and reworked everything.


In [2]:
#Load different datasets
xls = pd.ExcelFile(r'C:\Users\Pearc\OneDrive\Documents\Data Science\Springboard\SpringboardCapstone2\data\raw_data\India Premier League - Sold and Unsold -  Auction Prices Dec 2021 for 2022 season.xlsx')
Sold21IPL = pd.read_excel(xls, 'Sold Dec 2021')
UnSold21IPL = pd.read_excel(xls, 'Unsold Dec 2021')


In [3]:
#Look at dataset of players sold in IPL auction in Dec 2021
Sold21IPL.info()
Sold21IPL.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 204 entries, 0 to 203
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   PLAYER         204 non-null    object
 1   Country        204 non-null    object
 2   Team           204 non-null    object
 3   Type           204 non-null    object
 4   Auction_price  204 non-null    object
dtypes: object(5)
memory usage: 8.1+ KB


,PLAYER,Country,Team,Type,Auction_price
0,Deepak Chahar,India,Chennai Super Kings,Bowler,"₹14,00,00,000"
1,Ambati Rayudu,India,Chennai Super Kings,Wicket Keeper,"₹6,75,00,000"
2,Dwayne Bravo,West Indies,Chennai Super Kings,All-Rounder,"₹4,40,00,000"
3,Shivam Dube,India,Chennai Super Kings,All-Rounder,"₹4,00,00,000"
4,Chris Jordan,England,Chennai Super Kings,All-Rounder,"₹3,60,00,000"


Auction prices are in Indian rupees, and different numbering system:
One Crore = 10m Rupees = £99720 (so multiply rupees by 0.009972 to convert to pounds)
One lakh = 100,000 Rupees

In [4]:
#Remove rupee symbol
Sold21IPL['Auction_price'] = Sold21IPL['Auction_price'].apply(lambda x: x.replace('₹','')).apply(lambda x: x.replace(',','')).astype(int)


In [5]:
#Looks good
Sold21IPL.head()

,PLAYER,Country,Team,Type,Auction_price
0,Deepak Chahar,India,Chennai Super Kings,Bowler,140000000
1,Ambati Rayudu,India,Chennai Super Kings,Wicket Keeper,67500000
2,Dwayne Bravo,West Indies,Chennai Super Kings,All-Rounder,44000000
3,Shivam Dube,India,Chennai Super Kings,All-Rounder,40000000
4,Chris Jordan,England,Chennai Super Kings,All-Rounder,36000000


In [6]:
Sold21IPL.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 204 entries, 0 to 203
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   PLAYER         204 non-null    object
 1   Country        204 non-null    object
 2   Team           204 non-null    object
 3   Type           204 non-null    object
 4   Auction_price  204 non-null    int32 
dtypes: int32(1), object(4)
memory usage: 7.3+ KB


In [7]:
#Convert auction price to pounds sterling
Sold21IPL['Auc_price_pounds'] = Sold21IPL['Auction_price']*0.009972
Sold21IPL.rename(columns = {'Auction_price':'Auction_Base_price', 'Auc_price_pounds':'Auc_Base_price_pounds'}, inplace = True)
Sold21IPL.head()

,PLAYER,Country,Team,Type,Auction_Base_price,Auc_Base_price_pounds
0,Deepak Chahar,India,Chennai Super Kings,Bowler,140000000,1396080.0
1,Ambati Rayudu,India,Chennai Super Kings,Wicket Keeper,67500000,673110.0
2,Dwayne Bravo,West Indies,Chennai Super Kings,All-Rounder,44000000,438768.0
3,Shivam Dube,India,Chennai Super Kings,All-Rounder,40000000,398880.0
4,Chris Jordan,England,Chennai Super Kings,All-Rounder,36000000,358992.0


In [8]:
#And those that weren't sold
UnSold21IPL.info()
UnSold21IPL.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 396 entries, 0 to 395
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   PLAYER      396 non-null    object 
 1   Country     396 non-null    object 
 2   Team        396 non-null    object 
 3   Type        396 non-null    object 
 4   Base Price  396 non-null    object 
 5   Unnamed: 5  0 non-null      float64
 6   Unnamed: 6  1 non-null      object 
dtypes: float64(1), object(6)
memory usage: 21.8+ KB


,PLAYER,Country,Team,Type,Base Price,Unnamed: 5,Unnamed: 6
0,Suresh Raina,India,Unsold,Batsman,"₹2,00,00,000",NaN,NaN
1,Steve Smith,Australia,Unsold,Batsman,"₹2,00,00,000",NaN,NaN
2,Shakib Al Hasan,Bangladesh,Unsold,All-Rounder,"₹2,00,00,000",NaN,NaN
3,Adil Rashid,England,Unsold,Bowler,"₹2,00,00,000",NaN,NaN
4,Imran Tahir,South Africa,Unsold,Bowler,"₹2,00,00,000",NaN,NaN


In [9]:
#Remove rupee symbol and convert to pounds
UnSold21IPL['Base Price'] = UnSold21IPL['Base Price'].apply(lambda x: x.replace('₹','')).apply(lambda x: x.replace(',','')).astype(int)
UnSold21IPL['Base_Price_Pounds'] = UnSold21IPL['Base Price']*0.009972
UnSold21IPL.head()

,PLAYER,Country,Team,Type,Base Price,Unnamed: 5,Unnamed: 6,Base_Price_Pounds
0,Suresh Raina,India,Unsold,Batsman,20000000,NaN,NaN,199440.0
1,Steve Smith,Australia,Unsold,Batsman,20000000,NaN,NaN,199440.0
2,Shakib Al Hasan,Bangladesh,Unsold,All-Rounder,20000000,NaN,NaN,199440.0
3,Adil Rashid,England,Unsold,Bowler,20000000,NaN,NaN,199440.0
4,Imran Tahir,South Africa,Unsold,Bowler,20000000,NaN,NaN,199440.0


In [10]:
#Rename columns and drop two unnamed ones
UnSold21IPL.rename(columns = {'Base Price':'Auction_Base_price', 'Base_Price_Pounds':'Auc_Base_price_pounds'}, inplace = True)
UnSold21IPL.drop(['Unnamed: 5', 'Unnamed: 6'], axis=1, inplace=True)
UnSold21IPL.head()

,PLAYER,Country,Team,Type,Auction_Base_price,Auc_Base_price_pounds
0,Suresh Raina,India,Unsold,Batsman,20000000,199440.0
1,Steve Smith,Australia,Unsold,Batsman,20000000,199440.0
2,Shakib Al Hasan,Bangladesh,Unsold,All-Rounder,20000000,199440.0
3,Adil Rashid,England,Unsold,Bowler,20000000,199440.0
4,Imran Tahir,South Africa,Unsold,Bowler,20000000,199440.0


In [11]:
#Concatenate the sold and unsold IPL players
SoldUnSold21 = pd.concat([Sold21IPL, UnSold21IPL], ignore_index=True)
SoldUnSold21.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   PLAYER                 600 non-null    object 
 1   Country                600 non-null    object 
 2   Team                   600 non-null    object 
 3   Type                   600 non-null    object 
 4   Auction_Base_price     600 non-null    int32  
 5   Auc_Base_price_pounds  600 non-null    float64
dtypes: float64(1), int32(1), object(4)
memory usage: 25.9+ KB


In [12]:
#Find key stats and characteristics of SoldUnSold dataset
SoldUnSold21.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
PLAYER,600,596,Shubham Singh,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Country,600,18,India,375,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Team,600,11,Unsold,396,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Type,600,4,All-Rounder,232,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Auction_Base_price,600.0,NaN,NaN,NaN,11779166.666667,22460593.931859,2000000.0,2000000.0,2000000.0,7500000.0,152500000.0
Auc_Base_price_pounds,600.0,NaN,NaN,NaN,117461.85,223977.042688,19944.0,19944.0,19944.0,74790.0,1520730.0


In [13]:
#Assess levels of missing data. Use pd.concat to put into tabular form
missing = pd.concat([SoldUnSold21.isnull().sum(), 100 * SoldUnSold21.isnull().mean()], axis=1)
missing.columns=['count', '%']
missing.sort_values(by='count')

,count,%
PLAYER,0,0.0
Country,0,0.0
Team,0,0.0
Type,0,0.0
Auction_Base_price,0,0.0
Auc_Base_price_pounds,0,0.0


Now that we've got the auction prices for the 2021 auction (re the players who performed in 2022), let's do the same for the December 2022 auctions for the forthcoming 2023 season

In [14]:
#Load different datasets
xls = pd.ExcelFile(r'C:\Users\Pearc\OneDrive\Documents\Data Science\Springboard\SpringboardCapstone2\data\raw_data\new_data_Mar_23\IPL_Auction_Data_23_Dec_2022.xlsx')
Sold22IPL = pd.read_excel(xls, 'Sold 2022')
UnSold22IPL = pd.read_excel(xls, 'Unsold 2022')


In [15]:
#Look at dataset of players sold in IPL auction in Dec 2022
Sold22IPL.info()
Sold22IPL.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80 entries, 0 to 79
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   PLAYER         80 non-null     object
 1   Country        80 non-null     object
 2   Team           80 non-null     object
 3   Type           80 non-null     object
 4   Auction_price  80 non-null     object
dtypes: object(5)
memory usage: 3.2+ KB


,PLAYER,Country,Team,Type,Auction_price
0,Ben Stokes,Overseas,Chennai Super Kings,All-Rounder,"₹16,25,00,000"
1,Kyle Jamieson,Overseas,Chennai Super Kings,Bowler,"₹1,00,00,000"
2,Nishant Sindhu,Indian,Chennai Super Kings,All-Rounder,"₹60,00,000"
3,Ajinkya Rahane,Indian,Chennai Super Kings,Batter,"₹50,00,000"
4,Bhagath Varma,Indian,Chennai Super Kings,All-Rounder,"₹20,00,000"


In [16]:
#Remove rupee symbol and convert auction price to pounds sterling
Sold22IPL['Auction_price'] = Sold22IPL['Auction_price'].apply(lambda x: x.replace('₹','')).apply(lambda x: x.replace(',','')).astype(int)
Sold22IPL['Auc_price_pounds'] = Sold22IPL['Auction_price']*0.009972
Sold22IPL.rename(columns = {'Auction_price':'Auction_Base_price', 'Auc_price_pounds':'Auc_Base_price_pounds'}, inplace = True)
Sold22IPL.head()


,PLAYER,Country,Team,Type,Auction_Base_price,Auc_Base_price_pounds
0,Ben Stokes,Overseas,Chennai Super Kings,All-Rounder,162500000,1620450.0
1,Kyle Jamieson,Overseas,Chennai Super Kings,Bowler,10000000,99720.0
2,Nishant Sindhu,Indian,Chennai Super Kings,All-Rounder,6000000,59832.0
3,Ajinkya Rahane,Indian,Chennai Super Kings,Batter,5000000,49860.0
4,Bhagath Varma,Indian,Chennai Super Kings,All-Rounder,2000000,19944.0


In [17]:
#And those that weren't sold
UnSold22IPL.info()
UnSold22IPL.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 325 entries, 0 to 324
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   PLAYER      325 non-null    object
 1   Country     325 non-null    object
 2   Team        325 non-null    object
 3   Type        325 non-null    object
 4   Base_price  325 non-null    object
dtypes: object(5)
memory usage: 12.8+ KB


,PLAYER,Country,Team,Type,Base_price
0,Tom Banton,Overseas,Unsold,Wicket-Keeper,"₹2,00,00,000"
1,Chris Jordan,Overseas,Unsold,Bowler,"₹2,00,00,000"
2,Adam Milne,Overseas,Unsold,Bowler,"₹2,00,00,000"
3,Travis Head,Overseas,Unsold,Batter,"₹2,00,00,000"
4,Rassie Van Der Dussen,Overseas,Unsold,Batter,"₹2,00,00,000"


In [18]:
#Remove rupee symbol and convert to pounds
UnSold22IPL['Base_price'] = UnSold22IPL['Base_price'].apply(lambda x: x.replace('₹','')).apply(lambda x: x.replace(',','')).astype(int)
UnSold22IPL['Base_Price_Pounds'] = UnSold22IPL['Base_price']*0.009972
UnSold22IPL.head()

,PLAYER,Country,Team,Type,Base_price,Base_Price_Pounds
0,Tom Banton,Overseas,Unsold,Wicket-Keeper,20000000,199440.0
1,Chris Jordan,Overseas,Unsold,Bowler,20000000,199440.0
2,Adam Milne,Overseas,Unsold,Bowler,20000000,199440.0
3,Travis Head,Overseas,Unsold,Batter,20000000,199440.0
4,Rassie Van Der Dussen,Overseas,Unsold,Batter,20000000,199440.0


In [19]:
#Rename columns
UnSold22IPL.rename(columns = {'Base_price':'Auction_Base_price', 'Base_Price_Pounds':'Auc_Base_price_pounds'}, inplace = True)
UnSold22IPL.head()

,PLAYER,Country,Team,Type,Auction_Base_price,Auc_Base_price_pounds
0,Tom Banton,Overseas,Unsold,Wicket-Keeper,20000000,199440.0
1,Chris Jordan,Overseas,Unsold,Bowler,20000000,199440.0
2,Adam Milne,Overseas,Unsold,Bowler,20000000,199440.0
3,Travis Head,Overseas,Unsold,Batter,20000000,199440.0
4,Rassie Van Der Dussen,Overseas,Unsold,Batter,20000000,199440.0


In [20]:
#Concatenate the sold and unsold Dec 2022 IPL players
SoldUnSold22 = pd.concat([Sold22IPL, UnSold22IPL], ignore_index=True)
SoldUnSold22.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 405 entries, 0 to 404
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   PLAYER                 405 non-null    object 
 1   Country                405 non-null    object 
 2   Team                   405 non-null    object 
 3   Type                   405 non-null    object 
 4   Auction_Base_price     405 non-null    int32  
 5   Auc_Base_price_pounds  405 non-null    float64
dtypes: float64(1), int32(1), object(4)
memory usage: 17.5+ KB


In [21]:
#Concatenate all 2021 and 2022 IPL players (sold and unsold)
SoldUnSold21_22 = pd.concat([SoldUnSold21, SoldUnSold22], ignore_index=True)
SoldUnSold21_22.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1005 entries, 0 to 1004
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   PLAYER                 1005 non-null   object 
 1   Country                1005 non-null   object 
 2   Team                   1005 non-null   object 
 3   Type                   1005 non-null   object 
 4   Auction_Base_price     1005 non-null   int32  
 5   Auc_Base_price_pounds  1005 non-null   float64
dtypes: float64(1), int32(1), object(4)
memory usage: 43.3+ KB


In [22]:
#Look for duplicates
duplicates = SoldUnSold21_22[SoldUnSold21_22.duplicated('PLAYER')]
print(duplicates)

              PLAYER Country                 Team         Type  \
478      Amit Mishra   India               Unsold       Bowler   
498      Lalit Yadav   India               Unsold       Bowler   
511    Shubham Singh   India               Unsold       Bowler   
536    Shivam Sharma   India               Unsold  All-Rounder   
603   Ajinkya Rahane  Indian  Chennai Super Kings       Batter   
...              ...     ...                  ...          ...   
986   Shubhang Hegde  Indian               Unsold  All-Rounder   
992   Shubham Sharma  Indian               Unsold  All-Rounder   
993       Amit Yadav  Indian               Unsold  All-Rounder   
994         Amit Ali  Indian               Unsold  All-Rounder   
1000     Dhruv Patel  Indian               Unsold  All-Rounder   

      Auction_Base_price  Auc_Base_price_pounds  
478              2000000                19944.0  
498              2000000                19944.0  
511              2000000                19944.0  
536    

In [23]:
#223 duplicate rows - check out one or two
SoldUnSold21_22[SoldUnSold21_22['PLAYER']=='Amit Mishra']

,PLAYER,Country,Team,Type,Auction_Base_price,Auc_Base_price_pounds
216,Amit Mishra,India,Unsold,Bowler,15000000,149580.0
478,Amit Mishra,India,Unsold,Bowler,2000000,19944.0
629,Amit Mishra,Indian,Lucknow Super Giants,Bowler,5000000,49860.0


In [24]:
SoldUnSold21_22[SoldUnSold21_22['PLAYER']=='Lalit Yadav']

,PLAYER,Country,Team,Type,Auction_Base_price,Auc_Base_price_pounds
32,Lalit Yadav,India,Delhi Capitals,All-Rounder,6500000,64818.0
498,Lalit Yadav,India,Unsold,Bowler,2000000,19944.0


In [25]:
#The unsold rows have far less data value/use, so will try to isolate those that 
#represent a team and thus an auction sale value
Keep = duplicates[duplicates['Team']!='Unsold']
Keep.head()

,PLAYER,Country,Team,Type,Auction_Base_price,Auc_Base_price_pounds
603,Ajinkya Rahane,Indian,Chennai Super Kings,Batter,5000000,49860.0
605,Ajay Mandal,Indian,Chennai Super Kings,All-Rounder,2000000,19944.0
608,Rilee Rossouw,Overseas,Delhi Capitals,Batter,46000000,458712.0
609,Manish Pandey,Indian,Delhi Capitals,Batter,24000000,239328.0
611,Ishant Sharma,Indian,Delhi Capitals,Bowler,5000000,49860.0


In [26]:
#Turns out to be only 47 of those representing at team/auction sale
Keep.shape

(47, 6)

In [27]:
#Drop duplicates Player names from 21 and 22 data (so drop the full 223 duplicates, with a view to reinstating the valuable ones)
SoldUnSold21_22 = SoldUnSold21_22.drop_duplicates(subset=['PLAYER'])
SoldUnSold21_22.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 782 entries, 0 to 1004
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   PLAYER                 782 non-null    object 
 1   Country                782 non-null    object 
 2   Team                   782 non-null    object 
 3   Type                   782 non-null    object 
 4   Auction_Base_price     782 non-null    int32  
 5   Auc_Base_price_pounds  782 non-null    float64
dtypes: float64(1), int32(1), object(4)
memory usage: 39.7+ KB


In [28]:
#Add back the retained duplicates ('Keep')
SoldUnSold21_22 = pd.concat([SoldUnSold21_22, Keep], ignore_index=True)
SoldUnSold21_22.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 829 entries, 0 to 828
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   PLAYER                 829 non-null    object 
 1   Country                829 non-null    object 
 2   Team                   829 non-null    object 
 3   Type                   829 non-null    object 
 4   Auction_Base_price     829 non-null    int32  
 5   Auc_Base_price_pounds  829 non-null    float64
dtypes: float64(1), int32(1), object(4)
memory usage: 35.7+ KB


In [29]:
#Check for duplicates again
duplicates = SoldUnSold21_22[SoldUnSold21_22.duplicated('PLAYER')]
print(duplicates)

                  PLAYER   Country                         Team  \
782       Ajinkya Rahane    Indian          Chennai Super Kings   
783          Ajay Mandal    Indian          Chennai Super Kings   
784        Rilee Rossouw  Overseas               Delhi Capitals   
785        Manish Pandey    Indian               Delhi Capitals   
786        Ishant Sharma    Indian               Delhi Capitals   
787          Shivam Mavi    Indian               Gujarat Titans   
788          K.S. Bharat    Indian               Gujarat Titans   
789         Mohit Sharma    Indian               Gujarat Titans   
790          Odean Smith  Overseas               Gujarat Titans   
791          Urvil Patel    Indian               Gujarat Titans   
792      Shakib Al Hasan  Overseas        Kolkata Knight Riders   
793          David Wiese  Overseas        Kolkata Knight Riders   
794        N. Jagadeesan    Indian        Kolkata Knight Riders   
795        Vaibhav Arora    Indian        Kolkata Knight Rider

In [30]:
#As expected, it's the last 47 we've just added back in
duplicates.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47 entries, 782 to 828
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   PLAYER                 47 non-null     object 
 1   Country                47 non-null     object 
 2   Team                   47 non-null     object 
 3   Type                   47 non-null     object 
 4   Auction_Base_price     47 non-null     int32  
 5   Auc_Base_price_pounds  47 non-null     float64
dtypes: float64(1), int32(1), object(4)
memory usage: 2.4+ KB


In [31]:
#Just check a couple of rows to be sure
SoldUnSold21_22[SoldUnSold21_22['PLAYER']=='Amit Mishra']

,PLAYER,Country,Team,Type,Auction_Base_price,Auc_Base_price_pounds
216,Amit Mishra,India,Unsold,Bowler,15000000,149580.0
801,Amit Mishra,Indian,Lucknow Super Giants,Bowler,5000000,49860.0


In [32]:
SoldUnSold21_22[SoldUnSold21_22['PLAYER']=='Mandeep Singh']

,PLAYER,Country,Team,Type,Auction_Base_price,Auc_Base_price_pounds
31,Mandeep Singh,India,Delhi Capitals,Batsman,11000000,109692.0
796,Mandeep Singh,Indian,Kolkata Knight Riders,Batter,5000000,49860.0


In [33]:
#Create a subset of all duplicates, including the first instance (94 in all)
duplicates2 = SoldUnSold21_22[SoldUnSold21_22.duplicated('PLAYER', keep=False)]
duplicates2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 94 entries, 14 to 828
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   PLAYER                 94 non-null     object 
 1   Country                94 non-null     object 
 2   Team                   94 non-null     object 
 3   Type                   94 non-null     object 
 4   Auction_Base_price     94 non-null     int32  
 5   Auc_Base_price_pounds  94 non-null     float64
dtypes: float64(1), int32(1), object(4)
memory usage: 4.8+ KB


In [34]:
#Remove those from subset that represent Unsold values - so of less use
duplicates2=duplicates2[duplicates2['Team']!='Unsold']
duplicates2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 66 entries, 14 to 828
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   PLAYER                 66 non-null     object 
 1   Country                66 non-null     object 
 2   Team                   66 non-null     object 
 3   Type                   66 non-null     object 
 4   Auction_Base_price     66 non-null     int32  
 5   Auc_Base_price_pounds  66 non-null     float64
dtypes: float64(1), int32(1), object(4)
memory usage: 3.4+ KB


In [35]:
#Remove all 94 duplicated values from full dataset (including first instances)
SoldUnSold21_22 = SoldUnSold21_22.drop_duplicates(subset=['PLAYER'], keep=False)
SoldUnSold21_22.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 735 entries, 0 to 781
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   PLAYER                 735 non-null    object 
 1   Country                735 non-null    object 
 2   Team                   735 non-null    object 
 3   Type                   735 non-null    object 
 4   Auction_Base_price     735 non-null    int32  
 5   Auc_Base_price_pounds  735 non-null    float64
dtypes: float64(1), int32(1), object(4)
memory usage: 37.3+ KB


In [36]:
#Add back the retained and refined duplicates ('duplicates2')
SoldUnSold21_22 = pd.concat([SoldUnSold21_22, duplicates2], ignore_index=True)
SoldUnSold21_22.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 801 entries, 0 to 800
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   PLAYER                 801 non-null    object 
 1   Country                801 non-null    object 
 2   Team                   801 non-null    object 
 3   Type                   801 non-null    object 
 4   Auction_Base_price     801 non-null    int32  
 5   Auc_Base_price_pounds  801 non-null    float64
dtypes: float64(1), int32(1), object(4)
memory usage: 34.5+ KB


In [37]:
#Check for duplicates again (about 19 remaining)
duplicates = SoldUnSold21_22[SoldUnSold21_22.duplicated('PLAYER')]
print(duplicates)

               PLAYER   Country                   Team           Type  \
754    Ajinkya Rahane    Indian    Chennai Super Kings         Batter   
757     Manish Pandey    Indian         Delhi Capitals         Batter   
759       Shivam Mavi    Indian         Gujarat Titans         Bowler   
760       K.S. Bharat    Indian         Gujarat Titans  Wicket-Keeper   
762       Odean Smith  Overseas         Gujarat Titans    All-Rounder   
766     N. Jagadeesan    Indian  Kolkata Knight Riders  Wicket-Keeper   
767     Vaibhav Arora    Indian  Kolkata Knight Riders         Bowler   
768     Mandeep Singh    Indian  Kolkata Knight Riders         Batter   
771   Nicholas Pooran  Overseas   Lucknow Super Giants  Wicket-Keeper   
772       Daniel Sams  Overseas   Lucknow Super Giants    All-Rounder   
774  Romario Shepherd  Overseas   Lucknow Super Giants    All-Rounder   
776    Jaydev Unadkat    Indian   Lucknow Super Giants         Bowler   
779     Prerak Mankad    Indian   Lucknow Super Gia

In [38]:
#Keep the second duplicates, because they represent the more recent (2022) auction data
SoldUnSold21_22 = SoldUnSold21_22.drop_duplicates(subset=['PLAYER'], keep='last', ignore_index=True)
SoldUnSold21_22.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 782 entries, 0 to 781
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   PLAYER                 782 non-null    object 
 1   Country                782 non-null    object 
 2   Team                   782 non-null    object 
 3   Type                   782 non-null    object 
 4   Auction_Base_price     782 non-null    int32  
 5   Auc_Base_price_pounds  782 non-null    float64
dtypes: float64(1), int32(1), object(4)
memory usage: 33.7+ KB


In [39]:
SoldUnSold21_22.head(10)

,PLAYER,Country,Team,Type,Auction_Base_price,Auc_Base_price_pounds
0,Deepak Chahar,India,Chennai Super Kings,Bowler,140000000,1396080.0
1,Ambati Rayudu,India,Chennai Super Kings,Wicket Keeper,67500000,673110.0
2,Dwayne Bravo,West Indies,Chennai Super Kings,All-Rounder,44000000,438768.0
3,Shivam Dube,India,Chennai Super Kings,All-Rounder,40000000,398880.0
4,Chris Jordan,England,Chennai Super Kings,All-Rounder,36000000,358992.0
5,Robin Uthappa,India,Chennai Super Kings,Batsman,20000000,199440.0
6,Mitchell Santner,New Zealand,Chennai Super Kings,All-Rounder,19000000,189468.0
7,Adam Milne,New Zealand,Chennai Super Kings,Bowler,19000000,189468.0
8,Rajvardhan Hangargekar,India,Chennai Super Kings,All-Rounder,15000000,149580.0
9,Prashant Solanki,India,Chennai Super Kings,Bowler,12000000,119664.0


In [40]:
#While working on data above, noticed that there are different descriptions within the Type column 
#for the same type of player
SoldUnSold21_22.Type.unique()

array(['Bowler', 'Wicket Keeper', 'All-Rounder', 'Batsman', 'Batter',
       'Wicket-Keeper'], dtype=object)

In [41]:
#Make changes using mask function
SoldUnSold21_22['Type'].mask(SoldUnSold21_22['Type'] == 'Wicket Keeper', 'Wicket-Keeper', inplace=True)
SoldUnSold21_22['Type'].mask(SoldUnSold21_22['Type'] == 'Batter', 'Batsman', inplace=True)
SoldUnSold21_22.Type.unique()

array(['Bowler', 'Wicket-Keeper', 'All-Rounder', 'Batsman'], dtype=object)

In [42]:
#Let's check the other category columns
SoldUnSold21_22.Country.unique()

array(['India', 'West Indies', 'England', 'New Zealand', 'Sri Lanka',
       'South Africa', 'Australia', 'Bangladesh', 'Afghanistan',
       'Singapore', 'New Zeland', 'Ireland', 'Zimbabwe', 'Scotland',
       'Nepal', 'USA', 'Indian', 'Namibia', 'Overseas'], dtype=object)

In [43]:
SoldUnSold21_22.Team.unique()

array(['Chennai Super Kings', 'Delhi Capitals', 'Gujarat Titans',
       'Kolkata Knight Riders', 'Lucknow Super Giants', 'Mumbai Indians',
       'Punjab Kings', 'Rajasthan Royals', 'Royal Challengers Bangalore',
       'Sunrisers Hyderabad', 'Unsold'], dtype=object)

In [44]:
#Change the New Zealand and India mistakes ('Overseas' isn't very helpful, but not sure what we can do about that)
SoldUnSold21_22['Country'].mask(SoldUnSold21_22['Country'] == 'Indian', 'India', inplace=True)
SoldUnSold21_22['Country'].mask(SoldUnSold21_22['Country'] == 'New Zeland', 'New Zealand', inplace=True)
SoldUnSold21_22.Country.unique()

array(['India', 'West Indies', 'England', 'New Zealand', 'Sri Lanka',
       'South Africa', 'Australia', 'Bangladesh', 'Afghanistan',
       'Singapore', 'Ireland', 'Zimbabwe', 'Scotland', 'Nepal', 'USA',
       'Namibia', 'Overseas'], dtype=object)

In [45]:
#Check overall price data - so means we will have c265 auction prices in total (782 total minus 517unsold)
SoldUnSold21_22['Team'].value_counts()

Unsold                         517
Rajasthan Royals                30
Sunrisers Hyderabad             30
Gujarat Titans                  27
Kolkata Knight Riders           27
Chennai Super Kings             26
Lucknow Super Giants            26
Punjab Kings                    26
Royal Challengers Bangalore     26
Mumbai Indians                  24
Delhi Capitals                  23
Name: Team, dtype: int64

In [46]:
#Bring in the other datasets
IPLBatStat21 = pd.read_excel(r'C:\Users\Pearc\OneDrive\Documents\Data Science\Springboard\SpringboardCapstone2\data\raw_data\new_data_Mar_23\2021_IPL_season_batting_data.xlsx')
IPLBowlStat21 = pd.read_excel(r'C:\Users\Pearc\OneDrive\Documents\Data Science\Springboard\SpringboardCapstone2\data\raw_data\new_data_Mar_23\2021_IPL_season_bowling_data.xlsx')
IPLMVPStat21 = pd.read_excel(r'C:\Users\Pearc\OneDrive\Documents\Data Science\Springboard\SpringboardCapstone2\data\raw_data\new_data_Mar_23\2021_IPL_season_MVP_data.xlsx')
IPLBatStat22 = pd.read_excel(r'C:\Users\Pearc\OneDrive\Documents\Data Science\Springboard\SpringboardCapstone2\data\raw_data\new_data_Mar_23\2022_IPL_season_batting_data.xlsx')
IPLBowlStat22 = pd.read_excel(r'C:\Users\Pearc\OneDrive\Documents\Data Science\Springboard\SpringboardCapstone2\data\raw_data\new_data_Mar_23\2022_IPL_season_bowling_data.xlsx')
IPLMVPStat22 = pd.read_excel(r'C:\Users\Pearc\OneDrive\Documents\Data Science\Springboard\SpringboardCapstone2\data\raw_data\new_data_Mar_23\2022_IPL_season_MVP_data.xlsx')
IntCrickSals = pd.read_excel(r'C:\Users\Pearc\OneDrive\Documents\Data Science\Springboard\SpringboardCapstone2\data\raw_data\International Mens Cricket Central Contract Salaries - 2021 22.xlsx')


When look through different IPL datasets below, looks like it could be worth renaming POS columns to indicate whether batting, bowling, MVP, as is a ranking system, and is another measure of worth or value.

In [47]:
#2021 batting stats for IPL - data types look good, but lots of blank rows
IPLBatStat21.info()
IPLBatStat21.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 593 entries, 0 to 592
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   POS     149 non-null    float64
 1   Player  149 non-null    object 
 2   Mat     148 non-null    float64
 3   Inns    149 non-null    float64
 4   NO      149 non-null    float64
 5   Runs    149 non-null    float64
 6   HS      149 non-null    object 
 7   Avg     149 non-null    float64
 8   BF      149 non-null    float64
 9   SR      149 non-null    float64
 10  100     149 non-null    float64
 11  50      149 non-null    float64
 12  4s      149 non-null    float64
 13  6s      149 non-null    float64
dtypes: float64(12), object(2)
memory usage: 65.0+ KB


,POS,Player,Mat,Inns,NO,Runs,HS,Avg,BF,SR,100,50,4s,6s
0,1.0,Ruturaj Gaikwad,16.0,16.0,2.0,635.0,101*,45.35,466.0,136.26,1.0,4.0,64.0,23.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2.0,Faf du Plessis,16.0,16.0,2.0,633.0,95*,45.21,458.0,138.20,0.0,6.0,60.0,23.0
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,3.0,KL Rahul,NaN,13.0,3.0,626.0,98*,62.60,451.0,138.80,0.0,6.0,48.0,30.0
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [48]:
#And thus lot of missing data
missing = pd.concat([IPLBatStat21.isnull().sum(), 100 * IPLBatStat21.isnull().mean()], axis=1)
missing.columns=['count', '%']
missing.sort_values(by='count')

,count,%
POS,444,74.873524
Player,444,74.873524
Inns,444,74.873524
NO,444,74.873524
Runs,444,74.873524
HS,444,74.873524
Avg,444,74.873524
BF,444,74.873524
SR,444,74.873524
100,444,74.873524


In [49]:
#Drop rows with missing values
IPLBatStat21.dropna(inplace=True)

In [50]:
#Looks better
IPLBatStat21.head(10)


,POS,Player,Mat,Inns,NO,Runs,HS,Avg,BF,SR,100,50,4s,6s
0,1.0,Ruturaj Gaikwad,16.0,16.0,2.0,635.0,101*,45.35,466.0,136.26,1.0,4.0,64.0,23.0
4,2.0,Faf du Plessis,16.0,16.0,2.0,633.0,95*,45.21,458.0,138.20,0.0,6.0,60.0,23.0
12,4.0,Shikhar Dhawan,16.0,16.0,1.0,587.0,92,39.13,471.0,124.62,0.0,3.0,63.0,16.0
16,5.0,Glenn Maxwell,15.0,14.0,2.0,513.0,78,42.75,356.0,144.10,0.0,6.0,48.0,21.0
20,6.0,Sanju Samson,14.0,14.0,2.0,484.0,119,40.33,354.0,136.72,1.0,2.0,45.0,17.0
24,7.0,Prithvi Shaw,15.0,15.0,0.0,479.0,82,31.93,301.0,159.13,0.0,4.0,56.0,18.0
28,8.0,Shubman Gill,17.0,17.0,0.0,478.0,57,28.11,402.0,118.90,0.0,3.0,50.0,12.0
32,9.0,Mayank Agarwal,12.0,12.0,1.0,441.0,99*,40.09,314.0,140.44,0.0,4.0,42.0,18.0
36,10.0,Rishabh Pant,16.0,16.0,4.0,419.0,58*,34.91,326.0,128.52,0.0,3.0,42.0,10.0
40,11.0,Devdutt Padikkal,14.0,14.0,1.0,411.0,101*,31.61,328.0,125.30,1.0,1.0,44.0,14.0


In [51]:
#And right number of players, etc
IPLBatStat21.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 147 entries, 0 to 592
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   POS     147 non-null    float64
 1   Player  147 non-null    object 
 2   Mat     147 non-null    float64
 3   Inns    147 non-null    float64
 4   NO      147 non-null    float64
 5   Runs    147 non-null    float64
 6   HS      147 non-null    object 
 7   Avg     147 non-null    float64
 8   BF      147 non-null    float64
 9   SR      147 non-null    float64
 10  100     147 non-null    float64
 11  50      147 non-null    float64
 12  4s      147 non-null    float64
 13  6s      147 non-null    float64
dtypes: float64(12), object(2)
memory usage: 17.2+ KB


In [52]:
#Reset index
IPLBatStat21 = IPLBatStat21.reset_index(drop = True)
IPLBatStat21.head(5)

,POS,Player,Mat,Inns,NO,Runs,HS,Avg,BF,SR,100,50,4s,6s
0,1.0,Ruturaj Gaikwad,16.0,16.0,2.0,635.0,101*,45.35,466.0,136.26,1.0,4.0,64.0,23.0
1,2.0,Faf du Plessis,16.0,16.0,2.0,633.0,95*,45.21,458.0,138.20,0.0,6.0,60.0,23.0
2,4.0,Shikhar Dhawan,16.0,16.0,1.0,587.0,92,39.13,471.0,124.62,0.0,3.0,63.0,16.0
3,5.0,Glenn Maxwell,15.0,14.0,2.0,513.0,78,42.75,356.0,144.10,0.0,6.0,48.0,21.0
4,6.0,Sanju Samson,14.0,14.0,2.0,484.0,119,40.33,354.0,136.72,1.0,2.0,45.0,17.0


In [53]:
#But no missing data now
missing = pd.concat([IPLBatStat21.isnull().sum(), 100 * IPLBatStat21.isnull().mean()], axis=1)
missing.columns=['count', '%']
missing.sort_values(by='count')

,count,%
POS,0,0.0
Player,0,0.0
Mat,0,0.0
Inns,0,0.0
NO,0,0.0
Runs,0,0.0
HS,0,0.0
Avg,0,0.0
BF,0,0.0
SR,0,0.0


In [54]:
#Same issue for 22 Batting data, I suspect
IPLBatStat22.info()
IPLBatStat22.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 643 entries, 0 to 642
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   POS     162 non-null    float64
 1   Player  162 non-null    object 
 2   Mat     162 non-null    float64
 3   Inns    162 non-null    float64
 4   NO      162 non-null    float64
 5   Runs    162 non-null    float64
 6   HS      162 non-null    object 
 7   Avg     162 non-null    object 
 8   BF      162 non-null    float64
 9   SR      162 non-null    float64
 10  100     162 non-null    float64
 11  50      162 non-null    float64
 12  4s      162 non-null    float64
 13  6s      162 non-null    float64
dtypes: float64(11), object(3)
memory usage: 70.5+ KB


,POS,Player,Mat,Inns,NO,Runs,HS,Avg,BF,SR,100,50,4s,6s
0,1.0,Jos Buttler,17.0,17.0,2.0,863.0,116,57.53,579.0,149.05,4.0,4.0,83.0,45.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2.0,K L Rahul,15.0,15.0,3.0,616.0,103*,51.33,455.0,135.38,2.0,4.0,45.0,30.0
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,3.0,Quinton De Kock,15.0,15.0,1.0,508.0,140*,36.29,341.0,148.97,1.0,3.0,47.0,23.0


In [55]:
#Drop rows with missing values
IPLBatStat22.dropna(inplace=True)
IPLBatStat22.head(10)

,POS,Player,Mat,Inns,NO,Runs,HS,Avg,BF,SR,100,50,4s,6s
0,1.0,Jos Buttler,17.0,17.0,2.0,863.0,116,57.53,579.0,149.05,4.0,4.0,83.0,45.0
5,2.0,K L Rahul,15.0,15.0,3.0,616.0,103*,51.33,455.0,135.38,2.0,4.0,45.0,30.0
9,3.0,Quinton De Kock,15.0,15.0,1.0,508.0,140*,36.29,341.0,148.97,1.0,3.0,47.0,23.0
13,4.0,Hardik Pandya,15.0,15.0,4.0,487.0,87*,44.27,371.0,131.26,0.0,4.0,49.0,12.0
17,5.0,Shubman Gill,16.0,16.0,2.0,483.0,96,34.5,365.0,132.32,0.0,4.0,51.0,11.0
21,6.0,David Miller,16.0,16.0,9.0,481.0,94*,68.71,337.0,142.72,0.0,2.0,32.0,23.0
25,7.0,Faf Du Plessis,16.0,16.0,1.0,468.0,96,31.2,367.0,127.52,0.0,3.0,49.0,13.0
29,8.0,Shikhar Dhawan,14.0,14.0,2.0,460.0,88*,38.33,375.0,122.66,0.0,3.0,47.0,12.0
33,9.0,Sanju Samson,17.0,17.0,1.0,458.0,55,28.63,312.0,146.79,0.0,2.0,43.0,26.0
37,10.0,Deepak Hooda,15.0,14.0,0.0,451.0,59,32.21,330.0,136.66,0.0,4.0,36.0,18.0


In [56]:
#Reset index
IPLBatStat22 = IPLBatStat22.reset_index(drop = True)
IPLBatStat22.head(10)

,POS,Player,Mat,Inns,NO,Runs,HS,Avg,BF,SR,100,50,4s,6s
0,1.0,Jos Buttler,17.0,17.0,2.0,863.0,116,57.53,579.0,149.05,4.0,4.0,83.0,45.0
1,2.0,K L Rahul,15.0,15.0,3.0,616.0,103*,51.33,455.0,135.38,2.0,4.0,45.0,30.0
2,3.0,Quinton De Kock,15.0,15.0,1.0,508.0,140*,36.29,341.0,148.97,1.0,3.0,47.0,23.0
3,4.0,Hardik Pandya,15.0,15.0,4.0,487.0,87*,44.27,371.0,131.26,0.0,4.0,49.0,12.0
4,5.0,Shubman Gill,16.0,16.0,2.0,483.0,96,34.5,365.0,132.32,0.0,4.0,51.0,11.0
5,6.0,David Miller,16.0,16.0,9.0,481.0,94*,68.71,337.0,142.72,0.0,2.0,32.0,23.0
6,7.0,Faf Du Plessis,16.0,16.0,1.0,468.0,96,31.2,367.0,127.52,0.0,3.0,49.0,13.0
7,8.0,Shikhar Dhawan,14.0,14.0,2.0,460.0,88*,38.33,375.0,122.66,0.0,3.0,47.0,12.0
8,9.0,Sanju Samson,17.0,17.0,1.0,458.0,55,28.63,312.0,146.79,0.0,2.0,43.0,26.0
9,10.0,Deepak Hooda,15.0,14.0,0.0,451.0,59,32.21,330.0,136.66,0.0,4.0,36.0,18.0


In [57]:
#Check missing values
missing = pd.concat([IPLBatStat22.isnull().sum(), 100 * IPLBatStat22.isnull().mean()], axis=1)
missing.columns=['count', '%']
missing.sort_values(by='count')

,count,%
POS,0,0.0
Player,0,0.0
Mat,0,0.0
Inns,0,0.0
NO,0,0.0
Runs,0,0.0
HS,0,0.0
Avg,0,0.0
BF,0,0.0
SR,0,0.0


It's also worth noting that HS (highest score) in this batting table is an object type, presumably because some numbers have an asterisk next to them, indicating that the highest score was "not out", so usually seen as having a bit more worth/value, due to the batsman not having lost his wicket. Therefore, will need to create another column to indicate whether or not the high score was not out. Will then have to apply this to other batting tables.

In [58]:
IPLBatStat21.dtypes

POS       float64
Player     object
Mat       float64
Inns      float64
NO        float64
Runs      float64
HS         object
Avg       float64
BF        float64
SR        float64
100       float64
50        float64
4s        float64
6s        float64
dtype: object

In [59]:
IPLBatStat21['HS'] = IPLBatStat21['HS'].astype('string')
HS_was_NO = []
for score in IPLBatStat21['HS']:
    if '*' in score:
        HS_was_NO.append(True)
    else:
        HS_was_NO.append(False)
IPLBatStat21['HS_was_NO'] = HS_was_NO
IPLBatStat21.head()

,POS,Player,Mat,Inns,NO,Runs,HS,Avg,BF,SR,100,50,4s,6s,HS_was_NO
0,1.0,Ruturaj Gaikwad,16.0,16.0,2.0,635.0,101*,45.35,466.0,136.26,1.0,4.0,64.0,23.0,True
1,2.0,Faf du Plessis,16.0,16.0,2.0,633.0,95*,45.21,458.0,138.20,0.0,6.0,60.0,23.0,True
2,4.0,Shikhar Dhawan,16.0,16.0,1.0,587.0,92,39.13,471.0,124.62,0.0,3.0,63.0,16.0,False
3,5.0,Glenn Maxwell,15.0,14.0,2.0,513.0,78,42.75,356.0,144.10,0.0,6.0,48.0,21.0,False
4,6.0,Sanju Samson,14.0,14.0,2.0,484.0,119,40.33,354.0,136.72,1.0,2.0,45.0,17.0,False


In [60]:
#Remove asterisk from High Score and convert type to float64
IPLBatStat21['HS'] = IPLBatStat21['HS'].apply(lambda x: x.replace('*','')).astype(float)
IPLBatStat21.info()
IPLBatStat21.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 147 entries, 0 to 146
Data columns (total 15 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   POS        147 non-null    float64
 1   Player     147 non-null    object 
 2   Mat        147 non-null    float64
 3   Inns       147 non-null    float64
 4   NO         147 non-null    float64
 5   Runs       147 non-null    float64
 6   HS         147 non-null    float64
 7   Avg        147 non-null    float64
 8   BF         147 non-null    float64
 9   SR         147 non-null    float64
 10  100        147 non-null    float64
 11  50         147 non-null    float64
 12  4s         147 non-null    float64
 13  6s         147 non-null    float64
 14  HS_was_NO  147 non-null    bool   
dtypes: bool(1), float64(13), object(1)
memory usage: 16.3+ KB


,POS,Player,Mat,Inns,NO,Runs,HS,Avg,BF,SR,100,50,4s,6s,HS_was_NO
0,1.0,Ruturaj Gaikwad,16.0,16.0,2.0,635.0,101.0,45.35,466.0,136.26,1.0,4.0,64.0,23.0,True
1,2.0,Faf du Plessis,16.0,16.0,2.0,633.0,95.0,45.21,458.0,138.20,0.0,6.0,60.0,23.0,True
2,4.0,Shikhar Dhawan,16.0,16.0,1.0,587.0,92.0,39.13,471.0,124.62,0.0,3.0,63.0,16.0,False
3,5.0,Glenn Maxwell,15.0,14.0,2.0,513.0,78.0,42.75,356.0,144.10,0.0,6.0,48.0,21.0,False
4,6.0,Sanju Samson,14.0,14.0,2.0,484.0,119.0,40.33,354.0,136.72,1.0,2.0,45.0,17.0,False


In [61]:
#And do the same for the 2022 data
IPLBatStat22['HS'] = IPLBatStat22['HS'].astype('string')
HS_was_NO = []
for score in IPLBatStat22['HS']:
    if '*' in score:
        HS_was_NO.append(True)
    else:
        HS_was_NO.append(False)
IPLBatStat22['HS_was_NO'] = HS_was_NO
IPLBatStat22['HS'] = IPLBatStat22['HS'].apply(lambda x: x.replace('*','')).astype(float)
IPLBatStat22.info()
IPLBatStat22.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162 entries, 0 to 161
Data columns (total 15 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   POS        162 non-null    float64
 1   Player     162 non-null    object 
 2   Mat        162 non-null    float64
 3   Inns       162 non-null    float64
 4   NO         162 non-null    float64
 5   Runs       162 non-null    float64
 6   HS         162 non-null    float64
 7   Avg        162 non-null    object 
 8   BF         162 non-null    float64
 9   SR         162 non-null    float64
 10  100        162 non-null    float64
 11  50         162 non-null    float64
 12  4s         162 non-null    float64
 13  6s         162 non-null    float64
 14  HS_was_NO  162 non-null    bool   
dtypes: bool(1), float64(12), object(2)
memory usage: 18.0+ KB


,POS,Player,Mat,Inns,NO,Runs,HS,Avg,BF,SR,100,50,4s,6s,HS_was_NO
0,1.0,Jos Buttler,17.0,17.0,2.0,863.0,116.0,57.53,579.0,149.05,4.0,4.0,83.0,45.0,False
1,2.0,K L Rahul,15.0,15.0,3.0,616.0,103.0,51.33,455.0,135.38,2.0,4.0,45.0,30.0,True
2,3.0,Quinton De Kock,15.0,15.0,1.0,508.0,140.0,36.29,341.0,148.97,1.0,3.0,47.0,23.0,True
3,4.0,Hardik Pandya,15.0,15.0,4.0,487.0,87.0,44.27,371.0,131.26,0.0,4.0,49.0,12.0,True
4,5.0,Shubman Gill,16.0,16.0,2.0,483.0,96.0,34.5,365.0,132.32,0.0,4.0,51.0,11.0,False


In [62]:
#And the averages column should be a float type in 2022 dataset, but found a problem at row 118
IPLBatStat22.iloc[118]


POS                   119.0
Player       Josh Hazlewood
Mat                    12.0
Inns                    5.0
NO                      5.0
Runs                   18.0
HS                      7.0
Avg                       -
BF                     26.0
SR                    69.23
100                     0.0
50                      0.0
4s                      1.0
6s                      0.0
HS_was_NO              True
Name: 118, dtype: object

In [63]:
#His batting average should be 18 runs divided by 5 innings = 3.6
IPLBatStat22.loc[118, 'Avg'] = 3.6
IPLBatStat22.iloc[118]

POS                   119.0
Player       Josh Hazlewood
Mat                    12.0
Inns                    5.0
NO                      5.0
Runs                   18.0
HS                      7.0
Avg                     3.6
BF                     26.0
SR                    69.23
100                     0.0
50                      0.0
4s                      1.0
6s                      0.0
HS_was_NO              True
Name: 118, dtype: object

In [64]:
IPLBatStat22['Avg'] = IPLBatStat21['Avg'].astype(float)
IPLBatStat22.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162 entries, 0 to 161
Data columns (total 15 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   POS        162 non-null    float64
 1   Player     162 non-null    object 
 2   Mat        162 non-null    float64
 3   Inns       162 non-null    float64
 4   NO         162 non-null    float64
 5   Runs       162 non-null    float64
 6   HS         162 non-null    float64
 7   Avg        147 non-null    float64
 8   BF         162 non-null    float64
 9   SR         162 non-null    float64
 10  100        162 non-null    float64
 11  50         162 non-null    float64
 12  4s         162 non-null    float64
 13  6s         162 non-null    float64
 14  HS_was_NO  162 non-null    bool   
dtypes: bool(1), float64(13), object(1)
memory usage: 18.0+ KB


In [65]:
#Now have 147 non-null values in Avg compared to 162 previously
IPLBatStat22['Avg'].unique()

array([45.35, 45.21, 39.13, 42.75, 40.33, 31.93, 28.11, 40.09, 34.91,
       31.61, 28.92, 28.35, 29.46, 29.3 , 41.11, 25.5 , 22.64, 31.3 ,
       29.7 , 48.66, 44.33, 28.55, 36.28, 24.9 , 41.33, 30.62, 34.57,
       26.77, 28.75, 75.66, 22.3 , 24.37, 21.44, 38.2 , 26.14, 35.  ,
       16.  , 17.77, 15.5 , 21.85, 25.33, 30.2 , 30.  , 29.2 , 14.3 ,
       11.08, 14.55, 14.11, 24.8 , 57.5 , 16.28, 12.33, 16.33, 31.33,
       31.  , 11.62, 22.25, 14.16,  7.72, 10.37, 14.4 , 34.  , 13.4 ,
       16.25,  0.  ,  7.75,  8.42, 14.75, 11.6 , 18.66, 13.75, 49.  ,
        9.4 , 47.  , 14.66, 10.5 ,  8.4 , 20.5 ,  8.  , 11.33, 17.  ,
       26.  , 13.  , 25.  , 12.5 , 21.  , 20.  ,  9.5 , 18.  ,  3.2 ,
       15.  ,  7.5 ,  2.33,  2.8 ,  7.  ,  6.  ,  2.5 ,  2.  ,  3.  ,
        1.  ,   nan])

In [66]:
NullBool = pd.isnull(IPLBatStat22['Avg'])
 
# filtering data
# displaying data only with Avg = NaN
IPLBatStat22[NullBool]

,POS,Player,Mat,Inns,NO,Runs,HS,Avg,BF,SR,100,50,4s,6s,HS_was_NO
147,148.0,Abdul Samad,2.0,2.0,0.0,4.0,4.0,NaN,7.0,57.14,0.0,0.0,0.0,0.0,False
148,149.0,Abhijeet Tomar,1.0,1.0,0.0,4.0,4.0,NaN,8.0,50.00,0.0,0.0,1.0,0.0,False
149,150.0,Umran Malik,14.0,4.0,3.0,4.0,3.0,NaN,8.0,50.00,0.0,0.0,0.0,0.0,True
150,151.0,Mustafizur Rahman,8.0,1.0,1.0,3.0,3.0,NaN,5.0,60.00,0.0,0.0,0.0,0.0,True
151,152.0,Kumar Kartikeya Singh,4.0,1.0,0.0,3.0,3.0,NaN,5.0,60.00,0.0,0.0,0.0,0.0,False
152,153.0,Shivam Mavi,6.0,3.0,1.0,3.0,2.0,NaN,7.0,42.85,0.0,0.0,0.0,0.0,False
153,154.0,Harshit Rana,2.0,2.0,1.0,2.0,2.0,NaN,2.0,100.00,0.0,0.0,0.0,0.0,False
154,155.0,Navdeep Saini,2.0,1.0,0.0,2.0,2.0,NaN,2.0,100.00,0.0,0.0,0.0,0.0,False
155,156.0,Tristan Stubbs,2.0,2.0,0.0,2.0,2.0,NaN,4.0,50.00,0.0,0.0,0.0,0.0,False
156,157.0,Pradeep Sangwan,3.0,1.0,0.0,2.0,2.0,NaN,5.0,40.00,0.0,0.0,0.0,0.0,False


In [67]:
#Tried to loop through this or use lamda, but couldn't make it work, so changed averages manually

IPLBatStat22.at[147,'Avg']=2
IPLBatStat22.at[148,'Avg']=4
IPLBatStat22.at[149,'Avg']=1
IPLBatStat22.at[150,'Avg']=3
IPLBatStat22.at[151,'Avg']=3
IPLBatStat22.at[152,'Avg']=1
IPLBatStat22.at[153,'Avg']=1
IPLBatStat22.at[154,'Avg']=2
IPLBatStat22.at[155,'Avg']=1
IPLBatStat22.at[156,'Avg']=2
IPLBatStat22.at[157,'Avg']=2
IPLBatStat22.at[158,'Avg']=0.4
IPLBatStat22.at[159,'Avg']=0.22
IPLBatStat22.at[160,'Avg']=1
IPLBatStat22.at[161,'Avg']=1


In [68]:
IPLBatStat22.loc[147:162]
#NullBool = pd.isnull(IPLBatStat22['Avg'])
#IPLBatStat22[NullBool]

,POS,Player,Mat,Inns,NO,Runs,HS,Avg,BF,SR,100,50,4s,6s,HS_was_NO
147,148.0,Abdul Samad,2.0,2.0,0.0,4.0,4.0,2.00,7.0,57.14,0.0,0.0,0.0,0.0,False
148,149.0,Abhijeet Tomar,1.0,1.0,0.0,4.0,4.0,4.00,8.0,50.00,0.0,0.0,1.0,0.0,False
149,150.0,Umran Malik,14.0,4.0,3.0,4.0,3.0,1.00,8.0,50.00,0.0,0.0,0.0,0.0,True
150,151.0,Mustafizur Rahman,8.0,1.0,1.0,3.0,3.0,3.00,5.0,60.00,0.0,0.0,0.0,0.0,True
151,152.0,Kumar Kartikeya Singh,4.0,1.0,0.0,3.0,3.0,3.00,5.0,60.00,0.0,0.0,0.0,0.0,False
152,153.0,Shivam Mavi,6.0,3.0,1.0,3.0,2.0,1.00,7.0,42.85,0.0,0.0,0.0,0.0,False
153,154.0,Harshit Rana,2.0,2.0,1.0,2.0,2.0,1.00,2.0,100.00,0.0,0.0,0.0,0.0,False
154,155.0,Navdeep Saini,2.0,1.0,0.0,2.0,2.0,2.00,2.0,100.00,0.0,0.0,0.0,0.0,False
155,156.0,Tristan Stubbs,2.0,2.0,0.0,2.0,2.0,1.00,4.0,50.00,0.0,0.0,0.0,0.0,False
156,157.0,Pradeep Sangwan,3.0,1.0,0.0,2.0,2.0,2.00,5.0,40.00,0.0,0.0,0.0,0.0,False


In [69]:
IPLBatStat22.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162 entries, 0 to 161
Data columns (total 15 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   POS        162 non-null    float64
 1   Player     162 non-null    object 
 2   Mat        162 non-null    float64
 3   Inns       162 non-null    float64
 4   NO         162 non-null    float64
 5   Runs       162 non-null    float64
 6   HS         162 non-null    float64
 7   Avg        162 non-null    float64
 8   BF         162 non-null    float64
 9   SR         162 non-null    float64
 10  100        162 non-null    float64
 11  50         162 non-null    float64
 12  4s         162 non-null    float64
 13  6s         162 non-null    float64
 14  HS_was_NO  162 non-null    bool   
dtypes: bool(1), float64(13), object(1)
memory usage: 18.0+ KB


In [70]:
#Can drop POS cols from each dataset - simply gives who has the most runs (we can do this ourselves)
features = [ele for ele in IPLBatStat21.columns if ele != 'POS']

# Assign reduced column list to IPL Batting 21 stats
IPLBatStat21 = IPLBatStat21[features]

IPLBatStat21.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 147 entries, 0 to 146
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Player     147 non-null    object 
 1   Mat        147 non-null    float64
 2   Inns       147 non-null    float64
 3   NO         147 non-null    float64
 4   Runs       147 non-null    float64
 5   HS         147 non-null    float64
 6   Avg        147 non-null    float64
 7   BF         147 non-null    float64
 8   SR         147 non-null    float64
 9   100        147 non-null    float64
 10  50         147 non-null    float64
 11  4s         147 non-null    float64
 12  6s         147 non-null    float64
 13  HS_was_NO  147 non-null    bool   
dtypes: bool(1), float64(12), object(1)
memory usage: 15.2+ KB


In [71]:
#And for 2022 
features22 = [ele for ele in IPLBatStat22.columns if ele != 'POS']

# Assign reduced column list to IPL Batting 22 stats
IPLBatStat22 = IPLBatStat22[features22]

IPLBatStat22.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162 entries, 0 to 161
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Player     162 non-null    object 
 1   Mat        162 non-null    float64
 2   Inns       162 non-null    float64
 3   NO         162 non-null    float64
 4   Runs       162 non-null    float64
 5   HS         162 non-null    float64
 6   Avg        162 non-null    float64
 7   BF         162 non-null    float64
 8   SR         162 non-null    float64
 9   100        162 non-null    float64
 10  50         162 non-null    float64
 11  4s         162 non-null    float64
 12  6s         162 non-null    float64
 13  HS_was_NO  162 non-null    bool   
dtypes: bool(1), float64(12), object(1)
memory usage: 16.7+ KB


In [72]:
#Check out bowling data for 2021
IPLBowlStat21.info()
IPLBowlStat21.head(10)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 353 entries, 0 to 352
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   POS     89 non-null     float64       
 1   Player  89 non-null     object        
 2   Mat     89 non-null     float64       
 3   Inns    89 non-null     float64       
 4   Ov      89 non-null     float64       
 5   Runs    89 non-null     float64       
 6   Wkts    89 non-null     float64       
 7   BBI     89 non-null     datetime64[ns]
 8   Avg     89 non-null     float64       
 9   Econ    89 non-null     float64       
 10  SR      89 non-null     float64       
 11  4w      89 non-null     float64       
 12  5w      89 non-null     float64       
dtypes: datetime64[ns](1), float64(11), object(1)
memory usage: 36.0+ KB


,POS,Player,Mat,Inns,Ov,Runs,Wkts,BBI,Avg,Econ,SR,4w,5w
0,1.0,Harshal Patel,15.0,15.0,56.0,459.0,32.0,2027-05-01,14.34,8.14,10.56,1.0,1.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN
4,2.0,Avesh Khan,16.0,16.0,61.0,450.0,24.0,2013-03-01,18.75,7.37,15.25,0.0,0.0
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN
8,3.0,Jasprit Bumrah,14.0,14.0,55.0,410.0,21.0,2014-04-01,19.52,7.45,15.71,0.0,0.0
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN


In [73]:
#Drop rows with missing values
IPLBowlStat21.dropna(inplace=True)
IPLBowlStat21 = IPLBowlStat21.reset_index(drop = True)
IPLBowlStat21.head(10)

,POS,Player,Mat,Inns,Ov,Runs,Wkts,BBI,Avg,Econ,SR,4w,5w
0,1.0,Harshal Patel,15.0,15.0,56.0,459.0,32.0,2027-05-01,14.34,8.14,10.56,1.0,1.0
1,2.0,Avesh Khan,16.0,16.0,61.0,450.0,24.0,2013-03-01,18.75,7.37,15.25,0.0,0.0
2,3.0,Jasprit Bumrah,14.0,14.0,55.0,410.0,21.0,2014-04-01,19.52,7.45,15.71,0.0,0.0
3,4.0,Shardul Thakur,16.0,16.0,60.0,527.0,21.0,2019-03-01,25.09,8.80,17.09,0.0,0.0
4,5.0,Mohammad Shami,14.0,14.0,52.0,395.0,19.0,2015-03-01,20.78,7.50,16.63,0.0,0.0
5,6.0,Rashid Khan,14.0,14.0,56.0,375.0,18.0,2019-03-01,20.83,6.69,18.66,0.0,0.0
6,7.0,Arshdeep Singh,12.0,12.0,41.0,342.0,18.0,1932-05-01,19.00,8.27,13.77,0.0,1.0
7,8.0,Varun Chakaravarthy,17.0,17.0,68.0,448.0,18.0,2020-05-01,24.88,6.58,22.66,0.0,0.0
8,9.0,Yuzvendra Chahal,15.0,15.0,53.0,374.0,18.0,2025-04-01,20.77,7.05,17.66,0.0,0.0
9,10.0,Jason Holder,8.0,8.0,32.0,247.0,16.0,1952-04-01,15.43,7.75,11.93,1.0,0.0


In [74]:
#We can drop Position (based on total wickets taken) and BBI (best bowling in an innings because this is just a date record)
#ID cols to be dropped and use list comprehension
cols= ['POS', 'BBI']
featuresBowl21 = [ele for ele in IPLBowlStat21.columns if ele not in cols]

IPLBowlStat21 = IPLBowlStat21[featuresBowl21]

IPLBowlStat21.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89 entries, 0 to 88
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Player  89 non-null     object 
 1   Mat     89 non-null     float64
 2   Inns    89 non-null     float64
 3   Ov      89 non-null     float64
 4   Runs    89 non-null     float64
 5   Wkts    89 non-null     float64
 6   Avg     89 non-null     float64
 7   Econ    89 non-null     float64
 8   SR      89 non-null     float64
 9   4w      89 non-null     float64
 10  5w      89 non-null     float64
dtypes: float64(10), object(1)
memory usage: 7.8+ KB


In [75]:
#Check out bowling data for 2022
IPLBowlStat22.info()
IPLBowlStat22.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 410 entries, 0 to 409
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   POS     103 non-null    float64
 1   Player  103 non-null    object 
 2   Mat     103 non-null    float64
 3   Inns    103 non-null    float64
 4   Ov      103 non-null    float64
 5   Runs    103 non-null    float64
 6   Wkts    103 non-null    float64
 7   BBI     103 non-null    object 
 8   Avg     103 non-null    float64
 9   Econ    103 non-null    float64
 10  SR      103 non-null    float64
 11  4w      103 non-null    float64
 12  5w      103 non-null    float64
dtypes: float64(11), object(2)
memory usage: 41.8+ KB


,POS,Player,Mat,Inns,Ov,Runs,Wkts,BBI,Avg,Econ,SR,4w,5w
0,1.0,Yuzvendra Chahal,17.0,17.0,68.0,527.0,27.0,40/5,19.51,7.75,15.11,1.0,1.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2.0,Wanindu Hasaranga,16.0,16.0,57.0,430.0,26.0,2023-05-18 00:00:00,16.53,7.54,13.15,1.0,1.0
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,3.0,Kagiso Rabada,13.0,13.0,48.0,406.0,23.0,33/4,17.65,8.45,12.52,2.0,0.0


In [76]:
#Drop rows with missing values
IPLBowlStat22.dropna(inplace=True)
IPLBowlStat22 = IPLBowlStat22.reset_index(drop = True)
IPLBowlStat22.head(10)

,POS,Player,Mat,Inns,Ov,Runs,Wkts,BBI,Avg,Econ,SR,4w,5w
0,1.0,Yuzvendra Chahal,17.0,17.0,68.0,527.0,27.0,40/5,19.51,7.75,15.11,1.0,1.0
1,2.0,Wanindu Hasaranga,16.0,16.0,57.0,430.0,26.0,2023-05-18 00:00:00,16.53,7.54,13.15,1.0,1.0
2,3.0,Kagiso Rabada,13.0,13.0,48.0,406.0,23.0,33/4,17.65,8.45,12.52,2.0,0.0
3,4.0,Umran Malik,14.0,14.0,49.1,444.0,22.0,2023-05-25 00:00:00,20.18,9.03,13.40,1.0,1.0
4,5.0,Kuldeep Yadav,14.0,14.0,49.4,419.0,21.0,2023-04-14 00:00:00,19.95,8.43,14.19,2.0,0.0
5,6.0,Mohammad Shami,16.0,16.0,61.0,488.0,20.0,2023-03-25 00:00:00,24.40,8.00,18.30,0.0,0.0
6,7.0,Josh Hazlewood,12.0,12.0,46.3,377.0,20.0,2023-04-25 00:00:00,18.85,8.10,13.95,1.0,0.0
7,8.0,Rashid Khan,16.0,16.0,63.5,421.0,19.0,2023-04-24 00:00:00,22.15,6.59,20.15,1.0,0.0
8,9.0,Harshal Patel,15.0,15.0,53.3,410.0,19.0,34/4,21.57,7.66,16.89,1.0,0.0
9,10.0,Prasidh Krishna,17.0,17.0,66.3,551.0,19.0,2023-03-22 00:00:00,29.00,8.28,21.00,0.0,0.0


In [77]:
#And remove Pos and bbi cols for 2022 Bowling data
cols= ['POS', 'BBI']
featuresBowl22 = [ele for ele in IPLBowlStat22.columns if ele not in cols]

IPLBowlStat22 = IPLBowlStat22[featuresBowl22]

IPLBowlStat22.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Player  103 non-null    object 
 1   Mat     103 non-null    float64
 2   Inns    103 non-null    float64
 3   Ov      103 non-null    float64
 4   Runs    103 non-null    float64
 5   Wkts    103 non-null    float64
 6   Avg     103 non-null    float64
 7   Econ    103 non-null    float64
 8   SR      103 non-null    float64
 9   4w      103 non-null    float64
 10  5w      103 non-null    float64
dtypes: float64(10), object(1)
memory usage: 9.0+ KB


In [78]:
#And now MVP (most valued player) data for 2021
IPLMVPStat21.info()
IPLMVPStat21.head(10)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354 entries, 0 to 353
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   POS        89 non-null     float64
 1   Player     89 non-null     object 
 2   Pts        89 non-null     float64
 3   Mat        89 non-null     float64
 4   Wkts       89 non-null     float64
 5   Dots       89 non-null     float64
 6   4s         89 non-null     float64
 7   6s         89 non-null     float64
 8   Catches    89 non-null     float64
 9   Stumpings  89 non-null     float64
dtypes: float64(9), object(1)
memory usage: 27.8+ KB


,POS,Player,Pts,Mat,Wkts,Dots,4s,6s,Catches,Stumpings
0,1.0,Harshal Patel,264.5,15.0,32.0,122.0,6.0,2.0,3.0,0.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2.0,Ravindra Jadeja,263.0,16.0,13.0,104.0,19.0,9.0,13.0,0.0
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,3.0,Faf du Plessis,260.5,16.0,0.0,0.0,60.0,23.0,12.0,0.0


In [79]:
#Drop rows with missing values
IPLMVPStat21.dropna(inplace=True)
IPLMVPStat21 = IPLMVPStat21.reset_index(drop = True)
IPLMVPStat21.head(10)

,POS,Player,Pts,Mat,Wkts,Dots,4s,6s,Catches,Stumpings
0,1.0,Harshal Patel,264.5,15.0,32.0,122.0,6.0,2.0,3.0,0.0
1,2.0,Ravindra Jadeja,263.0,16.0,13.0,104.0,19.0,9.0,13.0,0.0
2,3.0,Faf du Plessis,260.5,16.0,0.0,0.0,60.0,23.0,12.0,0.0
3,4.0,Ruturaj Gaikwad,255.5,16.0,0.0,0.0,64.0,23.0,6.0,0.0
4,5.0,Lokesh Rahul,253.5,13.0,0.0,0.0,48.0,30.0,11.0,0.0
5,6.0,Avesh Khan,245.0,16.0,24.0,156.0,1.0,0.0,1.0,0.0
6,7.0,Glenn Maxwell,241.5,15.0,3.0,25.0,48.0,21.0,5.0,0.0
7,8.0,Shikhar Dhawan,236.0,16.0,0.0,0.0,63.0,16.0,9.0,0.0
8,9.0,Moeen Ali,232.5,15.0,6.0,50.0,31.0,19.0,7.0,0.0
9,10.0,Jasprit Bumrah,231.5,14.0,21.0,142.0,2.0,0.0,4.0,0.0


In [80]:
#The only information we need frm this is the Dots(dot balls bowled), catches and stumpings are the 
#rest is captured in the batting and bowling data already
cols=['Player', 'Dots', 'Catches', 'Stumpings']
IPLMVPStat21 = IPLMVPStat21[cols]
IPLMVPStat21.head(10)

,Player,Dots,Catches,Stumpings
0,Harshal Patel,122.0,3.0,0.0
1,Ravindra Jadeja,104.0,13.0,0.0
2,Faf du Plessis,0.0,12.0,0.0
3,Ruturaj Gaikwad,0.0,6.0,0.0
4,Lokesh Rahul,0.0,11.0,0.0
5,Avesh Khan,156.0,1.0,0.0
6,Glenn Maxwell,25.0,5.0,0.0
7,Shikhar Dhawan,0.0,9.0,0.0
8,Moeen Ali,50.0,7.0,0.0
9,Jasprit Bumrah,142.0,4.0,0.0


In [81]:
#And now MVP (most valued player) data for 2022
IPLMVPStat22.info()
IPLMVPStat22.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 754 entries, 0 to 753
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   POS        189 non-null    float64
 1   Player     189 non-null    object 
 2   Pts        189 non-null    float64
 3   Mat        189 non-null    float64
 4   Wkts       189 non-null    float64
 5   Dots       189 non-null    float64
 6   4s         189 non-null    float64
 7   6s         189 non-null    float64
 8   Catches    189 non-null    float64
 9   Run outs   189 non-null    float64
 10  Stumpings  189 non-null    float64
dtypes: float64(10), object(1)
memory usage: 64.9+ KB


,POS,Player,Pts,Mat,Wkts,Dots,4s,6s,Catches,Run outs,Stumpings
0,1.0,Jos Buttler,387.5,17.0,0.0,0.0,83.0,45.0,9.0,0.0,0.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2.0,Hardik Pandya,284.5,15.0,8.0,76.0,49.0,12.0,4.0,6.0,0.0
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,3.0,Andre Russell,281.0,14.0,17.0,58.0,18.0,32.0,2.0,1.5,0.0


In [82]:
#Drop rows with missing values
IPLMVPStat22.dropna(inplace=True)
IPLMVPStat22 = IPLMVPStat22.reset_index(drop = True)
IPLMVPStat22.head(10)

,POS,Player,Pts,Mat,Wkts,Dots,4s,6s,Catches,Run outs,Stumpings
0,1.0,Jos Buttler,387.5,17.0,0.0,0.0,83.0,45.0,9.0,0.0,0.0
1,2.0,Hardik Pandya,284.5,15.0,8.0,76.0,49.0,12.0,4.0,6.0,0.0
2,3.0,Andre Russell,281.0,14.0,17.0,58.0,18.0,32.0,2.0,1.5,0.0
3,4.0,Prasidh Krishna,273.0,17.0,19.0,200.0,0.0,0.0,2.0,1.5,0.0
4,5.0,Liam Livingstone,265.5,14.0,6.0,38.0,29.0,34.0,6.0,0.0,0.0
5,6.0,Wanindu Hasaranga,258.0,16.0,26.0,143.0,4.0,1.0,3.0,3.0,0.0
6,7.0,Mohammed Shami,252.5,16.0,20.0,172.0,0.0,0.0,3.0,3.0,0.0
7,8.0,Rashid Khan,250.0,16.0,19.0,127.0,3.0,9.0,7.0,0.0,0.0
8,9.0,Trent Boult,245.5,16.0,16.0,176.0,2.0,1.0,2.0,0.0,0.0
9,10.0,Sanju Samson,244.5,17.0,0.0,0.0,43.0,26.0,14.0,6.0,2.0


In [83]:
#Drop unnecessary columns (note that have added "run outs" data for 2022)
cols=['Player', 'Dots', 'Catches', 'Run outs', 'Stumpings']
IPLMVPStat22 = IPLMVPStat22[cols]
IPLMVPStat22.head(10)

,Player,Dots,Catches,Run outs,Stumpings
0,Jos Buttler,0.0,9.0,0.0,0.0
1,Hardik Pandya,76.0,4.0,6.0,0.0
2,Andre Russell,58.0,2.0,1.5,0.0
3,Prasidh Krishna,200.0,2.0,1.5,0.0
4,Liam Livingstone,38.0,6.0,0.0,0.0
5,Wanindu Hasaranga,143.0,3.0,3.0,0.0
6,Mohammed Shami,172.0,3.0,3.0,0.0
7,Rashid Khan,127.0,7.0,0.0,0.0
8,Trent Boult,176.0,2.0,0.0,0.0
9,Sanju Samson,0.0,14.0,6.0,2.0


In [84]:
#Let's see if we can join the three sets of 2021 data - start with bowing and MVP first

BowlMVPStats21 = pd.merge(IPLBowlStat21, IPLMVPStat21,
                              on = ["Player"],
                              how = "outer", suffixes=['_bowl', '_MVP'])
BowlMVPStats21.head()



,Player,Mat,Inns,Ov,Runs,Wkts,Avg,Econ,SR,4w,5w,Dots,Catches,Stumpings
0,Harshal Patel,15.0,15.0,56.0,459.0,32.0,14.34,8.14,10.56,1.0,1.0,122.0,3.0,0.0
1,Avesh Khan,16.0,16.0,61.0,450.0,24.0,18.75,7.37,15.25,0.0,0.0,156.0,1.0,0.0
2,Jasprit Bumrah,14.0,14.0,55.0,410.0,21.0,19.52,7.45,15.71,0.0,0.0,142.0,4.0,0.0
3,Shardul Thakur,16.0,16.0,60.0,527.0,21.0,25.09,8.80,17.09,0.0,0.0,137.0,2.0,0.0
4,Mohammad Shami,14.0,14.0,52.0,395.0,19.0,20.78,7.50,16.63,0.0,0.0,NaN,NaN,NaN


In [85]:
BowlMVPStats21.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 130 entries, 0 to 129
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Player     130 non-null    object 
 1   Mat        89 non-null     float64
 2   Inns       89 non-null     float64
 3   Ov         89 non-null     float64
 4   Runs       89 non-null     float64
 5   Wkts       89 non-null     float64
 6   Avg        89 non-null     float64
 7   Econ       89 non-null     float64
 8   SR         89 non-null     float64
 9   4w         89 non-null     float64
 10  5w         89 non-null     float64
 11  Dots       89 non-null     float64
 12  Catches    89 non-null     float64
 13  Stumpings  89 non-null     float64
dtypes: float64(13), object(1)
memory usage: 15.2+ KB


In [86]:
#And join with batting data to make full 2021 dataset
PerfStats21 = pd.merge(IPLBatStat21, BowlMVPStats21,
                              on = ["Player"],
                              how = "outer", suffixes=['_bat', '_BowlMVP'])
PerfStats21.head()


,Player,Mat_bat,Inns_bat,NO,Runs_bat,HS,Avg_bat,BF,SR_bat,100,...,Runs_BowlMVP,Wkts,Avg_BowlMVP,Econ,SR_BowlMVP,4w,5w,Dots,Catches,Stumpings
0,Ruturaj Gaikwad,16.0,16.0,2.0,635.0,101.0,45.35,466.0,136.26,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,6.0,0.0
1,Faf du Plessis,16.0,16.0,2.0,633.0,95.0,45.21,458.0,138.20,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,12.0,0.0
2,Shikhar Dhawan,16.0,16.0,1.0,587.0,92.0,39.13,471.0,124.62,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,9.0,0.0
3,Glenn Maxwell,15.0,14.0,2.0,513.0,78.0,42.75,356.0,144.10,0.0,...,135.0,3.0,45.0,8.43,32.0,0.0,0.0,25.0,5.0,0.0
4,Sanju Samson,14.0,14.0,2.0,484.0,119.0,40.33,354.0,136.72,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,7.0,4.0


In [87]:
PerfStats21.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 167 entries, 0 to 166
Data columns (total 27 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Player        167 non-null    object 
 1   Mat_bat       147 non-null    float64
 2   Inns_bat      147 non-null    float64
 3   NO            147 non-null    float64
 4   Runs_bat      147 non-null    float64
 5   HS            147 non-null    float64
 6   Avg_bat       147 non-null    float64
 7   BF            147 non-null    float64
 8   SR_bat        147 non-null    float64
 9   100           147 non-null    float64
 10  50            147 non-null    float64
 11  4s            147 non-null    float64
 12  6s            147 non-null    float64
 13  HS_was_NO     147 non-null    object 
 14  Mat_BowlMVP   89 non-null     float64
 15  Inns_BowlMVP  89 non-null     float64
 16  Ov            89 non-null     float64
 17  Runs_BowlMVP  89 non-null     float64
 18  Wkts          89 non-null     

In [88]:
PerfStats21.iloc[42]

Player          Jason Roy
Mat_bat               5.0
Inns_bat              5.0
NO                    0.0
Runs_bat            150.0
HS                   60.0
Avg_bat              30.0
BF                  121.0
SR_bat             123.96
100                   0.0
50                    1.0
4s                   21.0
6s                    1.0
HS_was_NO           False
Mat_BowlMVP           NaN
Inns_BowlMVP          NaN
Ov                    NaN
Runs_BowlMVP          NaN
Wkts                  NaN
Avg_BowlMVP           NaN
Econ                  NaN
SR_BowlMVP            NaN
4w                    NaN
5w                    NaN
Dots                  0.0
Catches               3.0
Stumpings             0.0
Name: 42, dtype: object

In [89]:
PerfStats21.loc[PerfStats21['Player'] == 'Harshal Patel'].T

,68
Player,Harshal Patel
Mat_bat,15.0
Inns_bat,8.0
NO,4.0
Runs_bat,59.0
HS,31.0
Avg_bat,14.75
BF,46.0
SR_bat,128.26
100,0.0


In [90]:
PerfStats21.loc[PerfStats21['Player'] == 'Deepak Chahar'].T

,136
Player,Deepak Chahar
Mat_bat,15.0
Inns_bat,2.0
NO,2.0
Runs_bat,1.0
HS,1.0
Avg_bat,0.0
BF,1.0
SR_bat,100.0
100,0.0


In [91]:
#We can drop the two Matches colums, as their relevant data is captured by the two Innings cols
cols= ['Mat_bat', 'Mat_BowlMVP']
featuresPerf21 = [ele for ele in PerfStats21.columns if ele not in cols]

PerfStats21 = PerfStats21[featuresPerf21]

PerfStats21.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 167 entries, 0 to 166
Data columns (total 25 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Player        167 non-null    object 
 1   Inns_bat      147 non-null    float64
 2   NO            147 non-null    float64
 3   Runs_bat      147 non-null    float64
 4   HS            147 non-null    float64
 5   Avg_bat       147 non-null    float64
 6   BF            147 non-null    float64
 7   SR_bat        147 non-null    float64
 8   100           147 non-null    float64
 9   50            147 non-null    float64
 10  4s            147 non-null    float64
 11  6s            147 non-null    float64
 12  HS_was_NO     147 non-null    object 
 13  Inns_BowlMVP  89 non-null     float64
 14  Ov            89 non-null     float64
 15  Runs_BowlMVP  89 non-null     float64
 16  Wkts          89 non-null     float64
 17  Avg_BowlMVP   89 non-null     float64
 18  Econ          89 non-null     

In [92]:
#Let's join up the 2022 data now
BowlMVPStats22 = pd.merge(IPLBowlStat22, IPLMVPStat22,
                              on = ["Player"],
                              how = "outer", suffixes=['_bowl', '_MVP'])
BowlMVPStats22.head()


,Player,Mat,Inns,Ov,Runs,Wkts,Avg,Econ,SR,4w,5w,Dots,Catches,Run outs,Stumpings
0,Yuzvendra Chahal,17.0,17.0,68.0,527.0,27.0,19.51,7.75,15.11,1.0,1.0,137.0,0.0,3.0,0.0
1,Wanindu Hasaranga,16.0,16.0,57.0,430.0,26.0,16.53,7.54,13.15,1.0,1.0,143.0,3.0,3.0,0.0
2,Kagiso Rabada,13.0,13.0,48.0,406.0,23.0,17.65,8.45,12.52,2.0,0.0,133.0,2.0,0.0,0.0
3,Umran Malik,14.0,14.0,49.1,444.0,22.0,20.18,9.03,13.40,1.0,1.0,141.0,5.0,0.0,0.0
4,Kuldeep Yadav,14.0,14.0,49.4,419.0,21.0,19.95,8.43,14.19,2.0,0.0,97.0,5.0,0.0,0.0


In [93]:
BowlMVPStats22.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 194 entries, 0 to 193
Data columns (total 15 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Player     194 non-null    object 
 1   Mat        103 non-null    float64
 2   Inns       103 non-null    float64
 3   Ov         103 non-null    float64
 4   Runs       103 non-null    float64
 5   Wkts       103 non-null    float64
 6   Avg        103 non-null    float64
 7   Econ       103 non-null    float64
 8   SR         103 non-null    float64
 9   4w         103 non-null    float64
 10  5w         103 non-null    float64
 11  Dots       189 non-null    float64
 12  Catches    189 non-null    float64
 13  Run outs   189 non-null    float64
 14  Stumpings  189 non-null    float64
dtypes: float64(14), object(1)
memory usage: 24.2+ KB


In [94]:
#And join with batting data to make full 2022 dataset
PerfStats22 = pd.merge(IPLBatStat22, BowlMVPStats22,
                              on = ["Player"],
                              how = "outer", suffixes=['_bat', '_BowlMVP'])
PerfStats22.info()
PerfStats22.head()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 208 entries, 0 to 207
Data columns (total 28 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Player        208 non-null    object 
 1   Mat_bat       162 non-null    float64
 2   Inns_bat      162 non-null    float64
 3   NO            162 non-null    float64
 4   Runs_bat      162 non-null    float64
 5   HS            162 non-null    float64
 6   Avg_bat       162 non-null    float64
 7   BF            162 non-null    float64
 8   SR_bat        162 non-null    float64
 9   100           162 non-null    float64
 10  50            162 non-null    float64
 11  4s            162 non-null    float64
 12  6s            162 non-null    float64
 13  HS_was_NO     162 non-null    object 
 14  Mat_BowlMVP   103 non-null    float64
 15  Inns_BowlMVP  103 non-null    float64
 16  Ov            103 non-null    float64
 17  Runs_BowlMVP  103 non-null    float64
 18  Wkts          103 non-null    

,Player,Mat_bat,Inns_bat,NO,Runs_bat,HS,Avg_bat,BF,SR_bat,100,...,Wkts,Avg_BowlMVP,Econ,SR_BowlMVP,4w,5w,Dots,Catches,Run outs,Stumpings
0,Jos Buttler,17.0,17.0,2.0,863.0,116.0,45.35,579.0,149.05,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,9.0,0.0,0.0
1,K L Rahul,15.0,15.0,3.0,616.0,103.0,45.21,455.0,135.38,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Quinton De Kock,15.0,15.0,1.0,508.0,140.0,39.13,341.0,148.97,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Hardik Pandya,15.0,15.0,4.0,487.0,87.0,42.75,371.0,131.26,0.0,...,8.0,27.75,7.27,22.87,0.0,0.0,76.0,4.0,6.0,0.0
4,Shubman Gill,16.0,16.0,2.0,483.0,96.0,40.33,365.0,132.32,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,7.0,0.0,0.0


In [95]:
#We can drop the two Matches colums, as before
cols= ['Mat_bat', 'Mat_BowlMVP']
featuresPerf22 = [ele for ele in PerfStats22.columns if ele not in cols]

PerfStats22 = PerfStats22[featuresPerf22]

PerfStats22.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 208 entries, 0 to 207
Data columns (total 26 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Player        208 non-null    object 
 1   Inns_bat      162 non-null    float64
 2   NO            162 non-null    float64
 3   Runs_bat      162 non-null    float64
 4   HS            162 non-null    float64
 5   Avg_bat       162 non-null    float64
 6   BF            162 non-null    float64
 7   SR_bat        162 non-null    float64
 8   100           162 non-null    float64
 9   50            162 non-null    float64
 10  4s            162 non-null    float64
 11  6s            162 non-null    float64
 12  HS_was_NO     162 non-null    object 
 13  Inns_BowlMVP  103 non-null    float64
 14  Ov            103 non-null    float64
 15  Runs_BowlMVP  103 non-null    float64
 16  Wkts          103 non-null    float64
 17  Avg_BowlMVP   103 non-null    float64
 18  Econ          103 non-null    

In [96]:
#Next join 21 and 22 batting data
PerfStats21_22 = pd.merge(PerfStats21, PerfStats22,
                              on = ["Player"],
                              how = "outer", suffixes=['_21', '_22'])
PerfStats21_22.head()

,Player,Inns_bat_21,NO_21,Runs_bat_21,HS_21,Avg_bat_21,BF_21,SR_bat_21,100_21,50_21,...,Wkts_22,Avg_BowlMVP_22,Econ_22,SR_BowlMVP_22,4w_22,5w_22,Dots_22,Catches_22,Run outs,Stumpings_22
0,Ruturaj Gaikwad,16.0,2.0,635.0,101.0,45.35,466.0,136.26,1.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,4.0,0.0,0.0
1,Faf du Plessis,16.0,2.0,633.0,95.0,45.21,458.0,138.20,0.0,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,4.0,3.0,0.0
2,Shikhar Dhawan,16.0,1.0,587.0,92.0,39.13,471.0,124.62,0.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,10.0,0.0,0.0
3,Glenn Maxwell,14.0,2.0,513.0,78.0,42.75,356.0,144.10,0.0,6.0,...,6.0,27.5,6.87,24.0,0.0,0.0,63.0,5.0,4.5,0.0
4,Sanju Samson,14.0,2.0,484.0,119.0,40.33,354.0,136.72,1.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,14.0,6.0,2.0


In [97]:
PerfStats21_22.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 252 entries, 0 to 251
Data columns (total 50 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Player           252 non-null    object 
 1   Inns_bat_21      147 non-null    float64
 2   NO_21            147 non-null    float64
 3   Runs_bat_21      147 non-null    float64
 4   HS_21            147 non-null    float64
 5   Avg_bat_21       147 non-null    float64
 6   BF_21            147 non-null    float64
 7   SR_bat_21        147 non-null    float64
 8   100_21           147 non-null    float64
 9   50_21            147 non-null    float64
 10  4s_21            147 non-null    float64
 11  6s_21            147 non-null    float64
 12  HS_was_NO_21     147 non-null    object 
 13  Inns_BowlMVP_21  89 non-null     float64
 14  Ov_21            89 non-null     float64
 15  Runs_BowlMVP_21  89 non-null     float64
 16  Wkts_21          89 non-null     float64
 17  Avg_BowlMVP_21  

In [98]:
#Check duplicates
duplicates = PerfStats21_22[PerfStats21_22.duplicated('Player')]
print(duplicates)


Empty DataFrame
Columns: [Player, Inns_bat_21, NO_21, Runs_bat_21, HS_21, Avg_bat_21, BF_21, SR_bat_21, 100_21, 50_21, 4s_21, 6s_21, HS_was_NO_21, Inns_BowlMVP_21, Ov_21, Runs_BowlMVP_21, Wkts_21, Avg_BowlMVP_21, Econ_21, SR_BowlMVP_21, 4w_21, 5w_21, Dots_21, Catches_21, Stumpings_21, Inns_bat_22, NO_22, Runs_bat_22, HS_22, Avg_bat_22, BF_22, SR_bat_22, 100_22, 50_22, 4s_22, 6s_22, HS_was_NO_22, Inns_BowlMVP_22, Ov_22, Runs_BowlMVP_22, Wkts_22, Avg_BowlMVP_22, Econ_22, SR_BowlMVP_22, 4w_22, 5w_22, Dots_22, Catches_22, Run outs, Stumpings_22]
Index: []

[0 rows x 50 columns]


In [99]:
PerfStats21_22.loc[PerfStats21_22['Player'] == 'Deepak Chahar'].T

,136
Player,Deepak Chahar
Inns_bat_21,2.0
NO_21,2.0
Runs_bat_21,1.0
HS_21,1.0
Avg_bat_21,0.0
BF_21,1.0
SR_bat_21,100.0
100_21,0.0
50_21,0.0


In [100]:
#Noticed 22 batting averages wrong datatype
PerfStats21_22['Avg_bat_22'] = PerfStats21_22['Avg_bat_22'].astype(float)
PerfStats21_22.dtypes['Avg_bat_22']

dtype('float64')

In [101]:
PerfStats21_22 = PerfStats21_22.fillna(0)
PerfStats21_22.head()

,Player,Inns_bat_21,NO_21,Runs_bat_21,HS_21,Avg_bat_21,BF_21,SR_bat_21,100_21,50_21,...,Wkts_22,Avg_BowlMVP_22,Econ_22,SR_BowlMVP_22,4w_22,5w_22,Dots_22,Catches_22,Run outs,Stumpings_22
0,Ruturaj Gaikwad,16.0,2.0,635.0,101.0,45.35,466.0,136.26,1.0,4.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,4.0,0.0,0.0
1,Faf du Plessis,16.0,2.0,633.0,95.0,45.21,458.0,138.20,0.0,6.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,4.0,3.0,0.0
2,Shikhar Dhawan,16.0,1.0,587.0,92.0,39.13,471.0,124.62,0.0,3.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,10.0,0.0,0.0
3,Glenn Maxwell,14.0,2.0,513.0,78.0,42.75,356.0,144.10,0.0,6.0,...,6.0,27.5,6.87,24.0,0.0,0.0,63.0,5.0,4.5,0.0
4,Sanju Samson,14.0,2.0,484.0,119.0,40.33,354.0,136.72,1.0,2.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,14.0,6.0,2.0


In [102]:
PerfStats21_22.loc[PerfStats21_22['Player'] == 'Deepak Chahar'].T

,136
Player,Deepak Chahar
Inns_bat_21,2.0
NO_21,2.0
Runs_bat_21,1.0
HS_21,1.0
Avg_bat_21,0.0
BF_21,1.0
SR_bat_21,100.0
100_21,0.0
50_21,0.0


In [103]:
PerfStats21_22.loc[PerfStats21_22['Player'] == 'Matthew Wade'].T

,181
Player,Matthew Wade
Inns_bat_21,0.0
NO_21,0.0
Runs_bat_21,0.0
HS_21,0.0
Avg_bat_21,0.0
BF_21,0.0
SR_bat_21,0.0
100_21,0.0
50_21,0.0


In [104]:
cols = ['Player', 'Bat_Inns_21_22', 'NOs_21_22', 'Run_21_21', 'HS_21', 'HS_was_NO_21', 'HS_22', 'HS_was_NO_22', 'Bat_Avg_21_22', 'BF_21_22', 'BatSR_21_22', '100s_21_22', '50s_21_22', '4s_21_22', '6s_21_22', 'Bowl_Inns_21_22', 'Ov_21_22', 'BowlRunsConc_21_22', 'Wkts_21_22', 'Bowl_Avg_21_22', 'Econ_21_22', 'BowlSR_21_22', '4w_21_22', '5w_21_22', 'Dots_21_22', 'Catches_21_22', 'Run outs', 'Stumpings_21_22']

PerfStats21_22['Bat_Inns_21_22'] = PerfStats21_22['Inns_bat_21'] + PerfStats21_22['Inns_bat_21']
PerfStats21_22['NOs_21_22'] = PerfStats21_22['NO_21'] + PerfStats21_22['NO_22']
PerfStats21_22['Run_21_21'] = PerfStats21_22['Runs_bat_21'] + PerfStats21_22['Runs_bat_22']
PerfStats21_22['Bat_Avg_21_22'] = (PerfStats21_22['Runs_bat_21'] + PerfStats21_22['Runs_bat_22'])/(PerfStats21_22['Inns_bat_21'] + PerfStats21_22['Inns_bat_22'])
PerfStats21_22['BF_21_22'] = PerfStats21_22['BF_21'] + PerfStats21_22['BF_22']
PerfStats21_22['BatSR_21_22'] = 100*(PerfStats21_22['Runs_bat_21'] + PerfStats21_22['Runs_bat_22'])/(PerfStats21_22['BF_21'] + PerfStats21_22['BF_22'])
PerfStats21_22['100s_21_22'] = PerfStats21_22['100_21'] + PerfStats21_22['100_22']
PerfStats21_22['50s_21_22'] = PerfStats21_22['50_21'] + PerfStats21_22['50_22']
PerfStats21_22['4s_21_22'] = PerfStats21_22['4s_21'] + PerfStats21_22['4s_22']
PerfStats21_22['6s_21_22'] = PerfStats21_22['6s_21'] + PerfStats21_22['6s_22']
PerfStats21_22['Bowl_Inns_21_22'] = PerfStats21_22['Inns_BowlMVP_21'] + PerfStats21_22['Inns_BowlMVP_22']
PerfStats21_22['Ov_21_22'] = PerfStats21_22['Ov_21'] + PerfStats21_22['Ov_22']
PerfStats21_22['BowlRunsConc_21_22'] = PerfStats21_22['Runs_BowlMVP_21'] + PerfStats21_22['Runs_BowlMVP_22']
PerfStats21_22['Wkts_21_22'] = PerfStats21_22['Wkts_21'] + PerfStats21_22['Wkts_22']
PerfStats21_22['Bowl_Avg_21_22'] = (PerfStats21_22['Runs_BowlMVP_21'] + PerfStats21_22['Runs_BowlMVP_22'])/(PerfStats21_22['Wkts_21'] + PerfStats21_22['Wkts_22'])
PerfStats21_22['Econ_21_22'] = (PerfStats21_22['Runs_BowlMVP_21'] + PerfStats21_22['Runs_BowlMVP_22'])/(PerfStats21_22['Ov_21'] + PerfStats21_22['Ov_22'])
PerfStats21_22['BowlSR_21_22'] = 6*(PerfStats21_22['Ov_21'] + PerfStats21_22['Ov_22'])/(PerfStats21_22['Wkts_21'] + PerfStats21_22['Wkts_22'])
PerfStats21_22['4w_21_22'] = PerfStats21_22['4w_21'] + PerfStats21_22['4w_22']
PerfStats21_22['5w_21_22'] = PerfStats21_22['5w_21'] + PerfStats21_22['5w_22']
PerfStats21_22['Dots_21_22'] =  PerfStats21_22['Dots_21'] + PerfStats21_22['Dots_22']
PerfStats21_22['Catches_21_22'] =  PerfStats21_22['Catches_21'] + PerfStats21_22['Catches_22']
PerfStats21_22['Stumpings_21_22'] =  PerfStats21_22['Stumpings_21'] + PerfStats21_22['Stumpings_22']

PerfStats21_22.columns

Index(['Player', 'Inns_bat_21', 'NO_21', 'Runs_bat_21', 'HS_21', 'Avg_bat_21',
       'BF_21', 'SR_bat_21', '100_21', '50_21', '4s_21', '6s_21',
       'HS_was_NO_21', 'Inns_BowlMVP_21', 'Ov_21', 'Runs_BowlMVP_21',
       'Wkts_21', 'Avg_BowlMVP_21', 'Econ_21', 'SR_BowlMVP_21', '4w_21',
       '5w_21', 'Dots_21', 'Catches_21', 'Stumpings_21', 'Inns_bat_22',
       'NO_22', 'Runs_bat_22', 'HS_22', 'Avg_bat_22', 'BF_22', 'SR_bat_22',
       '100_22', '50_22', '4s_22', '6s_22', 'HS_was_NO_22', 'Inns_BowlMVP_22',
       'Ov_22', 'Runs_BowlMVP_22', 'Wkts_22', 'Avg_BowlMVP_22', 'Econ_22',
       'SR_BowlMVP_22', '4w_22', '5w_22', 'Dots_22', 'Catches_22', 'Run outs',
       'Stumpings_22', 'Bat_Inns_21_22', 'NOs_21_22', 'Run_21_21',
       'Bat_Avg_21_22', 'BF_21_22', 'BatSR_21_22', '100s_21_22', '50s_21_22',
       '4s_21_22', '6s_21_22', 'Bowl_Inns_21_22', 'Ov_21_22',
       'BowlRunsConc_21_22', 'Wkts_21_22', 'Bowl_Avg_21_22', 'Econ_21_22',
       'BowlSR_21_22', '4w_21_22', '5w_21_22', 'D

In [105]:
PerfStats21_22.loc[PerfStats21_22['Player'] == 'Deepak Chahar']

,Player,Inns_bat_21,NO_21,Runs_bat_21,HS_21,Avg_bat_21,BF_21,SR_bat_21,100_21,50_21,...,BowlRunsConc_21_22,Wkts_21_22,Bowl_Avg_21_22,Econ_21_22,BowlSR_21_22,4w_21_22,5w_21_22,Dots_21_22,Catches_21_22,Stumpings_21_22
136,Deepak Chahar,2.0,2.0,1.0,1.0,0.0,1.0,100.0,0.0,0.0,...,451.0,14.0,32.214286,8.351852,23.142857,2.0,0.0,132.0,5.0,0.0


In [106]:
PerfStats21_22['Bat_Avg_21_22'][181]

15.7

In [107]:
PerfStats21_22 = PerfStats21_22[cols]
PerfStats21_22.columns

Index(['Player', 'Bat_Inns_21_22', 'NOs_21_22', 'Run_21_21', 'HS_21',
       'HS_was_NO_21', 'HS_22', 'HS_was_NO_22', 'Bat_Avg_21_22', 'BF_21_22',
       'BatSR_21_22', '100s_21_22', '50s_21_22', '4s_21_22', '6s_21_22',
       'Bowl_Inns_21_22', 'Ov_21_22', 'BowlRunsConc_21_22', 'Wkts_21_22',
       'Bowl_Avg_21_22', 'Econ_21_22', 'BowlSR_21_22', '4w_21_22', '5w_21_22',
       'Dots_21_22', 'Catches_21_22', 'Run outs', 'Stumpings_21_22'],
      dtype='object')

In [108]:
PerfStats21_22.head()

,Player,Bat_Inns_21_22,NOs_21_22,Run_21_21,HS_21,HS_was_NO_21,HS_22,HS_was_NO_22,Bat_Avg_21_22,BF_21_22,...,Wkts_21_22,Bowl_Avg_21_22,Econ_21_22,BowlSR_21_22,4w_21_22,5w_21_22,Dots_21_22,Catches_21_22,Run outs,Stumpings_21_22
0,Ruturaj Gaikwad,32.0,2.0,1003.0,101.0,True,99.0,False,33.433333,757.0,...,0.0,NaN,NaN,NaN,0.0,0.0,0.0,10.0,0.0,0.0
1,Faf du Plessis,32.0,2.0,633.0,95.0,True,0.0,0,39.562500,458.0,...,0.0,NaN,NaN,NaN,0.0,0.0,0.0,16.0,3.0,0.0
2,Shikhar Dhawan,32.0,3.0,1047.0,92.0,False,88.0,True,34.900000,846.0,...,0.0,NaN,NaN,NaN,0.0,0.0,0.0,19.0,0.0,0.0
3,Glenn Maxwell,28.0,4.0,814.0,78.0,False,55.0,False,30.148148,534.0,...,9.0,33.333333,7.5,26.666667,0.0,0.0,88.0,10.0,4.5,0.0
4,Sanju Samson,28.0,3.0,942.0,119.0,False,55.0,False,30.387097,666.0,...,0.0,NaN,NaN,NaN,0.0,0.0,0.0,21.0,6.0,6.0


In [109]:
PerfStats21_22.loc[PerfStats21_22['Player'] == 'Deepak Chahar'].T

,136
Player,Deepak Chahar
Bat_Inns_21_22,4.0
NOs_21_22,2.0
Run_21_21,1.0
HS_21,1.0
HS_was_NO_21,True
HS_22,0.0
HS_was_NO_22,0
Bat_Avg_21_22,0.5
BF_21_22,1.0


In [110]:
PerfStats21_22.loc[PerfStats21_22['Player'] == 'Faf du Plessis'].T

,1
Player,Faf du Plessis
Bat_Inns_21_22,32.0
NOs_21_22,2.0
Run_21_21,633.0
HS_21,95.0
HS_was_NO_21,True
HS_22,0.0
HS_was_NO_22,0
Bat_Avg_21_22,39.5625
BF_21_22,458.0


In [111]:
#Drop HS_was_NO (High Score was not out) columns - can see has NaNs that are hard to fix 
#and this feature has limited value compared to others
PerfStats21_22.drop(['HS_was_NO_21', 'HS_was_NO_22'], axis=1, inplace=True)
PerfStats21_22.head()

,Player,Bat_Inns_21_22,NOs_21_22,Run_21_21,HS_21,HS_22,Bat_Avg_21_22,BF_21_22,BatSR_21_22,100s_21_22,...,Wkts_21_22,Bowl_Avg_21_22,Econ_21_22,BowlSR_21_22,4w_21_22,5w_21_22,Dots_21_22,Catches_21_22,Run outs,Stumpings_21_22
0,Ruturaj Gaikwad,32.0,2.0,1003.0,101.0,99.0,33.433333,757.0,132.496697,1.0,...,0.0,NaN,NaN,NaN,0.0,0.0,0.0,10.0,0.0,0.0
1,Faf du Plessis,32.0,2.0,633.0,95.0,0.0,39.562500,458.0,138.209607,0.0,...,0.0,NaN,NaN,NaN,0.0,0.0,0.0,16.0,3.0,0.0
2,Shikhar Dhawan,32.0,3.0,1047.0,92.0,88.0,34.900000,846.0,123.758865,0.0,...,0.0,NaN,NaN,NaN,0.0,0.0,0.0,19.0,0.0,0.0
3,Glenn Maxwell,28.0,4.0,814.0,78.0,55.0,30.148148,534.0,152.434457,0.0,...,9.0,33.333333,7.5,26.666667,0.0,0.0,88.0,10.0,4.5,0.0
4,Sanju Samson,28.0,3.0,942.0,119.0,55.0,30.387097,666.0,141.441441,1.0,...,0.0,NaN,NaN,NaN,0.0,0.0,0.0,21.0,6.0,6.0


In [112]:
PerfStats21_22 = PerfStats21_22.fillna(0)
PerfStats21_22.head()

,Player,Bat_Inns_21_22,NOs_21_22,Run_21_21,HS_21,HS_22,Bat_Avg_21_22,BF_21_22,BatSR_21_22,100s_21_22,...,Wkts_21_22,Bowl_Avg_21_22,Econ_21_22,BowlSR_21_22,4w_21_22,5w_21_22,Dots_21_22,Catches_21_22,Run outs,Stumpings_21_22
0,Ruturaj Gaikwad,32.0,2.0,1003.0,101.0,99.0,33.433333,757.0,132.496697,1.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,10.0,0.0,0.0
1,Faf du Plessis,32.0,2.0,633.0,95.0,0.0,39.562500,458.0,138.209607,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,16.0,3.0,0.0
2,Shikhar Dhawan,32.0,3.0,1047.0,92.0,88.0,34.900000,846.0,123.758865,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,19.0,0.0,0.0
3,Glenn Maxwell,28.0,4.0,814.0,78.0,55.0,30.148148,534.0,152.434457,0.0,...,9.0,33.333333,7.5,26.666667,0.0,0.0,88.0,10.0,4.5,0.0
4,Sanju Samson,28.0,3.0,942.0,119.0,55.0,30.387097,666.0,141.441441,1.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,21.0,6.0,6.0


In [113]:
PerfStats21_22.loc[PerfStats21_22['Player'] == 'Faf du Plessis'].T

,1
Player,Faf du Plessis
Bat_Inns_21_22,32.0
NOs_21_22,2.0
Run_21_21,633.0
HS_21,95.0
HS_22,0.0
Bat_Avg_21_22,39.5625
BF_21_22,458.0
BatSR_21_22,138.209607
100s_21_22,0.0


In [114]:
PerfStats21_22.rename(columns = {'Run_21_21':'Run_21_22'}, inplace = True)

PerfStats21_22.head()

,Player,Bat_Inns_21_22,NOs_21_22,Run_21_22,HS_21,HS_22,Bat_Avg_21_22,BF_21_22,BatSR_21_22,100s_21_22,...,Wkts_21_22,Bowl_Avg_21_22,Econ_21_22,BowlSR_21_22,4w_21_22,5w_21_22,Dots_21_22,Catches_21_22,Run outs,Stumpings_21_22
0,Ruturaj Gaikwad,32.0,2.0,1003.0,101.0,99.0,33.433333,757.0,132.496697,1.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,10.0,0.0,0.0
1,Faf du Plessis,32.0,2.0,633.0,95.0,0.0,39.562500,458.0,138.209607,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,16.0,3.0,0.0
2,Shikhar Dhawan,32.0,3.0,1047.0,92.0,88.0,34.900000,846.0,123.758865,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,19.0,0.0,0.0
3,Glenn Maxwell,28.0,4.0,814.0,78.0,55.0,30.148148,534.0,152.434457,0.0,...,9.0,33.333333,7.5,26.666667,0.0,0.0,88.0,10.0,4.5,0.0
4,Sanju Samson,28.0,3.0,942.0,119.0,55.0,30.387097,666.0,141.441441,1.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,21.0,6.0,6.0


In [115]:
PerfStats21_22.shape

(252, 26)

In [116]:
#Make HS-NO changes into a function
def HS_NO_col(df, colname):
    '''Function to check which high scores were not-out scores
    and then create column with boolean True/False to indicate, 
    so that originaL HS col can have asterisks removed 
    and data type changed to float. Takes two arguments: dataframe 
    working on, and column name'''
    HS_was_NO = []
    for score in colname:
        if '*' in score:
            HS_was_NO.append(True)
        else:
            HS_was_NO.append(False)
    df['HS_was_NO'] = HS_was_NO
    df['colname'] = df['colname'].apply(lambda x: x.replace('*','')).astype(float)

In [117]:
IntCrickSals.info()
IntCrickSals.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 176 entries, 0 to 175
Data columns (total 15 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   Players                                176 non-null    object 
 1   Country                                176 non-null    object 
 2   Retainer Fee / contract / base salary  165 non-null    object 
 3   Test Fee                               171 non-null    object 
 4   ODI Fee                                167 non-null    object 
 5   T20 Fee                                164 non-null    object 
 6   Unnamed: 6                             0 non-null      float64
 7   Unnamed: 7                             0 non-null      float64
 8   Unnamed: 8                             0 non-null      float64
 9   Unnamed: 9                             0 non-null      float64
 10  Unnamed: 10                            0 non-null      float64
 11  Unname

,Players,Country,Retainer Fee / contract / base salary,Test Fee,ODI Fee,T20 Fee,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14
0,Babar Azam,Pakistan,1.25 million PKR (+ undisclosed Captain allowa...,"PKR 50,000","PKR 25,000","PKR 15,000",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,One Crore = 10m Rupees
1,Mohammad Rizwan,Pakistan,1.25 million PKR per month,"PKR 50,000","PKR 25,000","PKR 15,000",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"One lakh = 100,000 Rupees"
2,Hasan Ali,Pakistan,1.25 million PKR per month,"PKR 50,000","PKR 25,000","PKR 15,000",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Shaheen Afridi,Pakistan,1.25 million PKR per month,"PKR 50,000","PKR 25,000","PKR 15,000",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Azhar Ali,Pakistan,"937,500 PKRper month","PKR 40,000",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [118]:
#Drop unnamed columns
IntCrickSals.drop(['Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14'], axis=1, inplace=True)
IntCrickSals.head()


,Players,Country,Retainer Fee / contract / base salary,Test Fee,ODI Fee,T20 Fee
0,Babar Azam,Pakistan,1.25 million PKR (+ undisclosed Captain allowa...,"PKR 50,000","PKR 25,000","PKR 15,000"
1,Mohammad Rizwan,Pakistan,1.25 million PKR per month,"PKR 50,000","PKR 25,000","PKR 15,000"
2,Hasan Ali,Pakistan,1.25 million PKR per month,"PKR 50,000","PKR 25,000","PKR 15,000"
3,Shaheen Afridi,Pakistan,1.25 million PKR per month,"PKR 50,000","PKR 25,000","PKR 15,000"
4,Azhar Ali,Pakistan,"937,500 PKRper month","PKR 40,000",NaN,NaN


In [119]:
IntCrickSals.head(25)

,Players,Country,Retainer Fee / contract / base salary,Test Fee,ODI Fee,T20 Fee
0,Babar Azam,Pakistan,1.25 million PKR (+ undisclosed Captain allowa...,"PKR 50,000","PKR 25,000","PKR 15,000"
1,Mohammad Rizwan,Pakistan,1.25 million PKR per month,"PKR 50,000","PKR 25,000","PKR 15,000"
2,Hasan Ali,Pakistan,1.25 million PKR per month,"PKR 50,000","PKR 25,000","PKR 15,000"
3,Shaheen Afridi,Pakistan,1.25 million PKR per month,"PKR 50,000","PKR 25,000","PKR 15,000"
4,Azhar Ali,Pakistan,"937,500 PKRper month","PKR 40,000",NaN,NaN
5,Faheem Ashraf,Pakistan,"937,500 PKR per month","PKR 40,000","PKR 20,000","PKR 12,500"
6,Fakhar Zaman,Pakistan,"937,500 PKR per month","PKR 40,000","PKR 20,000","PKR 12,500"
7,Fawad Alam,Pakistan,"937,500 PKR per month","PKR 40,000",NaN,NaN
8,Shadab Khan,Pakistan,"937,500 PKR per month","PKR 40,000","PKR 20,000","PKR 12,500"
9,Yasir Shah,Pakistan,"937,500 PKR per month","PKR 40,000","PKR 20,000",NaN


In [120]:
IntCrickSals.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 176 entries, 0 to 175
Data columns (total 6 columns):
 #   Column                                 Non-Null Count  Dtype 
---  ------                                 --------------  ----- 
 0   Players                                176 non-null    object
 1   Country                                176 non-null    object
 2   Retainer Fee / contract / base salary  165 non-null    object
 3   Test Fee                               171 non-null    object
 4   ODI Fee                                167 non-null    object
 5   T20 Fee                                164 non-null    object
dtypes: object(6)
memory usage: 8.4+ KB


In [121]:
#How many countries are represented in dataset? Turns out to be 9
IntCrickSals.Country.unique()

array(['Pakistan', 'India', 'England', 'Australia', 'South Africa',
       'New Zealand', 'West Indies', 'Sri Lanka', 'Bangladesh'],
      dtype=object)

In [122]:
#View India players only - double-checked values and Jasprit Bumrah in partic - 
#the India base salary figures are annual, and JB's is a mistake to say monthly
India_players = IntCrickSals[IntCrickSals["Country"] == "India"]
print(India_players)

                Players Country Retainer Fee / contract / base salary  \
20          Virat Kohli   India                             7 Crores    
21     Rohit Sharma (C)   India                             7 Crores    
22       Jasprit Bumrah   India                    7 Crores per month   
23  Ravichandran Ashwin   India                             5 Crores    
24      Ravindra Jadeja   India                             5 Crores    
25         Rishabh Pant   India                              5 Crores   
26        KL Rahul (VC)   India                              5 Crores   
27       Mohammad Shami   India                              5 Crores   
28    Cheteshwar Pujara   India                              3 Crores   
29       Ajinkya Rahane   India                              3 Crores   
30           Axer Patel   India                              3 Crores   
31       Shradul Thakur   India                              3 Crores   
32         Sheryas Iyer   India                    

In [123]:
England_players = IntCrickSals[IntCrickSals["Country"] == "England"]
print(England_players)

              Players  Country          Retainer Fee / contract / base salary  \
47        Eoin Morgan  England  £925,000 (plus undisclosed captain allowance)   
48     James Anderson  England                                         925000   
49       Stuart Broad  England                                         925000   
50           Joe Root  England  £925,000 (Undisclosed test captain allowance)   
51         Ben Stokes  England                                         925000   
52  Jonathan Bairstow  England                                         925000   
53        Jos Buttler  England                                         925000   
54       Chris Woakes  England                                         925000   
55          Moeen Ali  England                                         925000   
56         Sam Curran  England                                         925000   
57       Jofra Archer  England                                         925000   
58          Jason Roy  Engla

In [124]:
print(IntCrickSals[IntCrickSals["Country"] == "England"])

              Players  Country          Retainer Fee / contract / base salary  \
47        Eoin Morgan  England  £925,000 (plus undisclosed captain allowance)   
48     James Anderson  England                                         925000   
49       Stuart Broad  England                                         925000   
50           Joe Root  England  £925,000 (Undisclosed test captain allowance)   
51         Ben Stokes  England                                         925000   
52  Jonathan Bairstow  England                                         925000   
53        Jos Buttler  England                                         925000   
54       Chris Woakes  England                                         925000   
55          Moeen Ali  England                                         925000   
56         Sam Curran  England                                         925000   
57       Jofra Archer  England                                         925000   
58          Jason Roy  Engla

In [125]:
Bangla_players = IntCrickSals[IntCrickSals["Country"] == "Bangladesh"]
Bangla_players.head()

,Players,Country,Retainer Fee / contract / base salary,Test Fee,ODI Fee,T20 Fee
155,Tamim Iqbal,Bangladesh,"$57,000","$7,100","$3,600","$2,400"
156,Soumya Sarkar,Bangladesh,NaN,"$7,100","$3,600","$2,400"
157,Shakib Al Hasan,Bangladesh,"$57,000","$7,100","$3,600","$2,400"
158,Shadman Islam,Bangladesh,NaN,"$7,100","$3,600","$2,400"
159,Sabbir Rahman,Bangladesh,NaN,"$7,100","$3,600","$2,400"


In [126]:
Aust_players = IntCrickSals[IntCrickSals["Country"] == "Australia"]
Aust_players.head()

,Players,Country,Retainer Fee / contract / base salary,Test Fee,ODI Fee,T20 Fee
67,Aaron Finch,Australia,$1.5 million,"$15,000","$18,000","$20,000"
68,Tim Paine,Australia,$1 million,"$15,000","$18,000","$20,000"
69,Steve Smith,Australia,$1 million,"$15,000","$18,000","$20,000"
70,Pat Cummins,Australia,"$850,000","$15,000","$18,000","$20,000"
71,Glenn Maxwell,Australia,"$850,000","$15,000","$18,000","$20,000"


In [127]:
WI_players = IntCrickSals[IntCrickSals["Country"] == "West Indies"]
WI_players.head()

,Players,Country,Retainer Fee / contract / base salary,Test Fee,ODI Fee,T20 Fee
119,Jason Holder,West Indies,$80000,"$6,500","$4,500","$3,000"
120,Kraigg Brathwaite,West Indies,$60000,"$6,500","$4,500","$3,000"
121,Jermaine Blackwood,West Indies,$60000,"$6,500","$4,500","$3,000"
122,Nkrumah Bonner,West Indies,$60000,"$6,500","$4,500","$3,000"
123,Rahkeem Cornwall,West Indies,$60000,"$6,500","$4,500","$3,000"


In [128]:
SouthA_players = IntCrickSals[IntCrickSals["Country"] == "South Africa"]
SouthA_players.head()

,Players,Country,Retainer Fee / contract / base salary,Test Fee,ODI Fee,T20 Fee
84,Temba Bavuma,South Africa,"$100,000","$4,500","$1,200",$800
85,Quinton de Kock,South Africa,"$350,000","$4,500","$1,200",$800
86,Dean Elgar,South Africa,"$100,000","$4,500","$1,200",$800
87,Beuran Hendricks,South Africa,"$100,000","$4,500","$1,200",$800
88,Reeza Hendricks,South Africa,"$100,000","$4,500","$1,200",$800


In [129]:
NZ_players = IntCrickSals[IntCrickSals["Country"] == "New Zealand"]
NZ_players.head()

,Players,Country,Retainer Fee / contract / base salary,Test Fee,ODI Fee,T20 Fee
100,Kane Williamson (C),New Zealand,"$440,000","$8,495","$3,682","$2,407"
101,Martin Guptill,New Zealand,"$350,000",NaN,"$3,682","$2,407"
102,Colin Munro,New Zealand,"$180,000",NaN,NaN,"$2,407"
103,Henry Nicholls,New Zealand,"$85,585","$8,495","$3,682",NaN
104,Ross Taylor,New Zealand,"$400,000","$8,495",NaN,"$2,407"


In [130]:
SriL_players = IntCrickSals[IntCrickSals["Country"] == "Sri Lanka"]
SriL_players.head()

,Players,Country,Retainer Fee / contract / base salary,Test Fee,ODI Fee,T20 Fee
137,Dimuth Karunaratne,Sri Lanka,"$100,000","$7,500","$5,500","$3,500"
138,Dinesh Chandimal,Sri Lanka,"$80,000","$7,500","$5,500","$3,500"
139,Dasun Shanaka,Sri Lanka,"$65,000","$7,500","$5,500","$3,500"
140,Suranga Lakmal,Sri Lanka,"$60,000","$7,500","$5,500","$3,500"
141,Lasith Embuldeniya,Sri Lanka,"$60,000","$7,500","$5,500","$3,500"


Looked at different salary levels - default currency is US dollars, but pounds for England, In Rupees for India, Pakistan rupees for Pakistan (and Pakistan also monthly, compared to annual for all others) 1.25milPKR = 4932
1 In Crore = £99590, 1 PKR = 0.0035

In [131]:
print(IntCrickSals[IntCrickSals["Country"] == "New Zealand"])

                  Players      Country Retainer Fee / contract / base salary  \
100   Kane Williamson (C)  New Zealand                              $440,000   
101        Martin Guptill  New Zealand                              $350,000   
102           Colin Munro  New Zealand                              $180,000   
103        Henry Nicholls  New Zealand                               $85,585   
104           Ross Taylor  New Zealand                              $400,000   
105        Daryl Mitchell  New Zealand                               $85,585   
106            Tom Latham  New Zealand                              $205,266   
107          Tom Blundell  New Zealand                               $85,585   
108           Tim Seifert  New Zealand                               $50,000   
109  Bradley John Watling  New Zealand                               $50,000   
110   Colin de Grandhomme  New Zealand                              $350,000   
111         James Neesham  New Zealand  

In [132]:
#To replace 'Crores' for India players - tried this approach below throughout whole dataframe column and ended up corrupting some other data - not sure why
IntCrickSals.loc[20:46, 'Retainer Fee / contract / base salary'] = IntCrickSals.loc[20:46, 'Retainer Fee / contract / base salary'].str.replace('Crores','0000000')
print(IntCrickSals[IntCrickSals["Country"] == "India"])

                Players Country Retainer Fee / contract / base salary  \
20          Virat Kohli   India                            7 0000000    
21     Rohit Sharma (C)   India                            7 0000000    
22       Jasprit Bumrah   India                   7 0000000 per month   
23  Ravichandran Ashwin   India                            5 0000000    
24      Ravindra Jadeja   India                            5 0000000    
25         Rishabh Pant   India                             5 0000000   
26        KL Rahul (VC)   India                             5 0000000   
27       Mohammad Shami   India                             5 0000000   
28    Cheteshwar Pujara   India                             3 0000000   
29       Ajinkya Rahane   India                             3 0000000   
30           Axer Patel   India                             3 0000000   
31       Shradul Thakur   India                             3 0000000   
32         Sheryas Iyer   India                    

In [133]:
#So to replace 'lakh'
IntCrickSals.loc[20:44, 'Test Fee'] = IntCrickSals.loc[20:44, 'Test Fee'].str.replace('lakh','00000')
IntCrickSals.loc[20:46, 'ODI Fee'] = IntCrickSals.loc[20:46, 'ODI Fee'].str.replace('lakh','00000')
IntCrickSals.loc[20:46, 'T20 Fee'] = IntCrickSals.loc[20:46, 'T20 Fee'].str.replace('Lakh', '00000')
print(IntCrickSals[IntCrickSals["Country"] == "India"])


                Players Country Retainer Fee / contract / base salary  \
20          Virat Kohli   India                            7 0000000    
21     Rohit Sharma (C)   India                            7 0000000    
22       Jasprit Bumrah   India                   7 0000000 per month   
23  Ravichandran Ashwin   India                            5 0000000    
24      Ravindra Jadeja   India                            5 0000000    
25         Rishabh Pant   India                             5 0000000   
26        KL Rahul (VC)   India                             5 0000000   
27       Mohammad Shami   India                             5 0000000   
28    Cheteshwar Pujara   India                             3 0000000   
29       Ajinkya Rahane   India                             3 0000000   
30           Axer Patel   India                             3 0000000   
31       Shradul Thakur   India                             3 0000000   
32         Sheryas Iyer   India                    

Looked up Deepak Chahar and Mayank Agarwal - mistake in their data - both on 15 lakh per test - https://www.timesofsports.com/cricket/indian-players-salary/

In [134]:
IntCrickSals.loc[45, 'Test Fee'] = 1500000
IntCrickSals.loc[46, 'Test Fee'] = 1500000
print(IntCrickSals[IntCrickSals["Country"] == "India"])

                Players Country Retainer Fee / contract / base salary  \
20          Virat Kohli   India                            7 0000000    
21     Rohit Sharma (C)   India                            7 0000000    
22       Jasprit Bumrah   India                   7 0000000 per month   
23  Ravichandran Ashwin   India                            5 0000000    
24      Ravindra Jadeja   India                            5 0000000    
25         Rishabh Pant   India                             5 0000000   
26        KL Rahul (VC)   India                             5 0000000   
27       Mohammad Shami   India                             5 0000000   
28    Cheteshwar Pujara   India                             3 0000000   
29       Ajinkya Rahane   India                             3 0000000   
30           Axer Patel   India                             3 0000000   
31       Shradul Thakur   India                             3 0000000   
32         Sheryas Iyer   India                    

In [135]:
IntCrickSals.loc[21, 'ODI Fee'] = 720000
IntCrickSals.loc[21, 'T20 Fee'] = 360000
print(IntCrickSals[IntCrickSals["Country"] == "India"])

                Players Country Retainer Fee / contract / base salary  \
20          Virat Kohli   India                            7 0000000    
21     Rohit Sharma (C)   India                            7 0000000    
22       Jasprit Bumrah   India                   7 0000000 per month   
23  Ravichandran Ashwin   India                            5 0000000    
24      Ravindra Jadeja   India                            5 0000000    
25         Rishabh Pant   India                             5 0000000   
26        KL Rahul (VC)   India                             5 0000000   
27       Mohammad Shami   India                             5 0000000   
28    Cheteshwar Pujara   India                             3 0000000   
29       Ajinkya Rahane   India                             3 0000000   
30           Axer Patel   India                             3 0000000   
31       Shradul Thakur   India                             3 0000000   
32         Sheryas Iyer   India                    

In [136]:
IntCrickSals.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 176 entries, 0 to 175
Data columns (total 6 columns):
 #   Column                                 Non-Null Count  Dtype 
---  ------                                 --------------  ----- 
 0   Players                                176 non-null    object
 1   Country                                176 non-null    object
 2   Retainer Fee / contract / base salary  165 non-null    object
 3   Test Fee                               171 non-null    object
 4   ODI Fee                                167 non-null    object
 5   T20 Fee                                164 non-null    object
dtypes: object(6)
memory usage: 8.4+ KB


In [137]:
null_series = pd.isnull(IntCrickSals['Retainer Fee / contract / base salary'])
IntCrickSals[null_series]

,Players,Country,Retainer Fee / contract / base salary,Test Fee,ODI Fee,T20 Fee
115,Lachlan Ferguson,New Zealand,NaN,"$8,495","$3,682","$2,407"
156,Soumya Sarkar,Bangladesh,NaN,"$7,100","$3,600","$2,400"
158,Shadman Islam,Bangladesh,NaN,"$7,100","$3,600","$2,400"
159,Sabbir Rahman,Bangladesh,NaN,"$7,100","$3,600","$2,400"
161,Nazmul Hossain Shanto,Bangladesh,NaN,"$7,100","$3,600","$2,400"
165,Mohmaad Saifudin,Bangladesh,NaN,"$7,100","$3,600","$2,400"
166,Mohammad Mithun Ali,Bangladesh,NaN,"$7,100","$3,600","$2,400"
171,Kamrul Islam Rabbi,Bangladesh,NaN,"$7,100","$3,600","$2,400"
173,Farhad Reza,Bangladesh,NaN,"$7,100","$3,600","$2,400"
174,Anamul Haque,Bangladesh,NaN,"$7,100","$3,600","$2,400"


In [138]:
null_series2 = pd.isnull(IntCrickSals['Test Fee'])
IntCrickSals[null_series2]

,Players,Country,Retainer Fee / contract / base salary,Test Fee,ODI Fee,T20 Fee
101,Martin Guptill,New Zealand,"$350,000",NaN,"$3,682","$2,407"
102,Colin Munro,New Zealand,"$180,000",NaN,NaN,"$2,407"
105,Daryl Mitchell,New Zealand,"$85,585",NaN,"$3,682","$2,407"
108,Tim Seifert,New Zealand,"$50,000",NaN,NaN,"$2,407"
111,James Neesham,New Zealand,"$350,000",NaN,"$3,682",$2407


In [139]:
null_series3 = pd.isnull(IntCrickSals['ODI Fee'])
IntCrickSals[null_series3]

,Players,Country,Retainer Fee / contract / base salary,Test Fee,ODI Fee,T20 Fee
4,Azhar Ali,Pakistan,"937,500 PKRper month","PKR 40,000",NaN,NaN
7,Fawad Alam,Pakistan,"937,500 PKR per month","PKR 40,000",NaN,NaN
10,Sarfaraz Ahmed,Pakistan,"562,500 PKR per month","PKR 37,500",NaN,"PKR 10,000"
18,"Imran Butt,",Pakistan,"300,500 PKR per month","PKR 30,000",NaN,NaN
102,Colin Munro,New Zealand,"$180,000",NaN,NaN,"$2,407"
104,Ross Taylor,New Zealand,"$400,000","$8,495",NaN,"$2,407"
108,Tim Seifert,New Zealand,"$50,000",NaN,NaN,"$2,407"
109,Bradley John Watling,New Zealand,"$50,000","$8,495",NaN,NaN
118,Neil Wagner,New Zealand,"$85,585","$8,495",NaN,NaN


In [140]:
null_series4 = pd.isnull(IntCrickSals['T20 Fee'])
IntCrickSals[null_series4]

,Players,Country,Retainer Fee / contract / base salary,Test Fee,ODI Fee,T20 Fee
4,Azhar Ali,Pakistan,"937,500 PKRper month","PKR 40,000",NaN,NaN
7,Fawad Alam,Pakistan,"937,500 PKR per month","PKR 40,000",NaN,NaN
9,Yasir Shah,Pakistan,"937,500 PKR per month","PKR 40,000","PKR 20,000",NaN
11,Imam-ul-Haq,Pakistan,"562,500 PKR per month","PKR 37,500","PKR 17,500",NaN
15,Nauman Ali,Pakistan,"562,500 PKR per month","PKR 37,500","PKR 17,500",NaN
16,Abid Ali,Pakistan,"562,500 PKR per month","PKR 37,500","PKR 17,500",NaN
18,"Imran Butt,",Pakistan,"300,500 PKR per month","PKR 30,000",NaN,NaN
103,Henry Nicholls,New Zealand,"$85,585","$8,495","$3,682",NaN
106,Tom Latham,New Zealand,"$205,266","$8,495","$3,682",NaN
109,Bradley John Watling,New Zealand,"$50,000","$8,495",NaN,NaN


In [141]:
IntCrickSals = IntCrickSals.fillna(0)


In [142]:
IntCrickSals[null_series4]

,Players,Country,Retainer Fee / contract / base salary,Test Fee,ODI Fee,T20 Fee
4,Azhar Ali,Pakistan,"937,500 PKRper month","PKR 40,000",0,0
7,Fawad Alam,Pakistan,"937,500 PKR per month","PKR 40,000",0,0
9,Yasir Shah,Pakistan,"937,500 PKR per month","PKR 40,000","PKR 20,000",0
11,Imam-ul-Haq,Pakistan,"562,500 PKR per month","PKR 37,500","PKR 17,500",0
15,Nauman Ali,Pakistan,"562,500 PKR per month","PKR 37,500","PKR 17,500",0
16,Abid Ali,Pakistan,"562,500 PKR per month","PKR 37,500","PKR 17,500",0
18,"Imran Butt,",Pakistan,"300,500 PKR per month","PKR 30,000",0,0
103,Henry Nicholls,New Zealand,"$85,585","$8,495","$3,682",0
106,Tom Latham,New Zealand,"$205,266","$8,495","$3,682",0
109,Bradley John Watling,New Zealand,"$50,000","$8,495",0,0


In [143]:
IntCrickSals['Base_sal_pounds'] = IntCrickSals['Retainer Fee / contract / base salary']
IntCrickSals.head()

,Players,Country,Retainer Fee / contract / base salary,Test Fee,ODI Fee,T20 Fee,Base_sal_pounds
0,Babar Azam,Pakistan,1.25 million PKR (+ undisclosed Captain allowa...,"PKR 50,000","PKR 25,000","PKR 15,000",1.25 million PKR (+ undisclosed Captain allowa...
1,Mohammad Rizwan,Pakistan,1.25 million PKR per month,"PKR 50,000","PKR 25,000","PKR 15,000",1.25 million PKR per month
2,Hasan Ali,Pakistan,1.25 million PKR per month,"PKR 50,000","PKR 25,000","PKR 15,000",1.25 million PKR per month
3,Shaheen Afridi,Pakistan,1.25 million PKR per month,"PKR 50,000","PKR 25,000","PKR 15,000",1.25 million PKR per month
4,Azhar Ali,Pakistan,"937,500 PKRper month","PKR 40,000",0,0,"937,500 PKRper month"


In [144]:
IntCrickSals.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 176 entries, 0 to 175
Data columns (total 7 columns):
 #   Column                                 Non-Null Count  Dtype 
---  ------                                 --------------  ----- 
 0   Players                                176 non-null    object
 1   Country                                176 non-null    object
 2   Retainer Fee / contract / base salary  176 non-null    object
 3   Test Fee                               176 non-null    object
 4   ODI Fee                                176 non-null    object
 5   T20 Fee                                176 non-null    object
 6   Base_sal_pounds                        176 non-null    object
dtypes: object(7)
memory usage: 9.8+ KB


In [145]:
print(IntCrickSals[IntCrickSals["Country"] == "England"])


              Players  Country          Retainer Fee / contract / base salary  \
47        Eoin Morgan  England  £925,000 (plus undisclosed captain allowance)   
48     James Anderson  England                                         925000   
49       Stuart Broad  England                                         925000   
50           Joe Root  England  £925,000 (Undisclosed test captain allowance)   
51         Ben Stokes  England                                         925000   
52  Jonathan Bairstow  England                                         925000   
53        Jos Buttler  England                                         925000   
54       Chris Woakes  England                                         925000   
55          Moeen Ali  England                                         925000   
56         Sam Curran  England                                         925000   
57       Jofra Archer  England                                         925000   
58          Jason Roy  Engla

In [146]:
null_series5 = pd.isnull(IntCrickSals['Base_sal_pounds'])
print(IntCrickSals[null_series5])

Empty DataFrame
Columns: [Players, Country, Retainer Fee / contract / base salary, Test Fee, ODI Fee, T20 Fee, Base_sal_pounds]
Index: []


In [147]:
IntCrickSals.loc[47:66, 'Base_sal_pounds'] = 925000
print(IntCrickSals[IntCrickSals["Country"] == "England"])

              Players  Country          Retainer Fee / contract / base salary  \
47        Eoin Morgan  England  £925,000 (plus undisclosed captain allowance)   
48     James Anderson  England                                         925000   
49       Stuart Broad  England                                         925000   
50           Joe Root  England  £925,000 (Undisclosed test captain allowance)   
51         Ben Stokes  England                                         925000   
52  Jonathan Bairstow  England                                         925000   
53        Jos Buttler  England                                         925000   
54       Chris Woakes  England                                         925000   
55          Moeen Ali  England                                         925000   
56         Sam Curran  England                                         925000   
57       Jofra Archer  England                                         925000   
58          Jason Roy  Engla

In [148]:
print(IntCrickSals[IntCrickSals["Country"] == "Bangladesh"])

                   Players     Country Retainer Fee / contract / base salary  \
155            Tamim Iqbal  Bangladesh                               $57,000   
156          Soumya Sarkar  Bangladesh                                     0   
157        Shakib Al Hasan  Bangladesh                               $57,000   
158          Shadman Islam  Bangladesh                                     0   
159          Sabbir Rahman  Bangladesh                                     0   
160          Rubel Hossain  Bangladesh                               $43,000   
161  Nazmul Hossain Shanto  Bangladesh                                     0   
162        Mushfiqur Rahim  Bangladesh                               $57,000   
163       Mosaddek Hossain  Bangladesh                               $14,000   
164          Mominul Haque  Bangladesh                               $30,000   
165       Mohmaad Saifudin  Bangladesh                                     0   
166    Mohammad Mithun Ali  Bangladesh  

In [149]:
null_series6 = pd.isnull(IntCrickSals['Base_sal_pounds'])
IntCrickSals[null_series6]

,Players,Country,Retainer Fee / contract / base salary,Test Fee,ODI Fee,T20 Fee,Base_sal_pounds


In [150]:
IntCrickSals['Base_sal_pounds'] = IntCrickSals['Base_sal_pounds'].fillna(0)
IntCrickSals.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 176 entries, 0 to 175
Data columns (total 7 columns):
 #   Column                                 Non-Null Count  Dtype 
---  ------                                 --------------  ----- 
 0   Players                                176 non-null    object
 1   Country                                176 non-null    object
 2   Retainer Fee / contract / base salary  176 non-null    object
 3   Test Fee                               176 non-null    object
 4   ODI Fee                                176 non-null    object
 5   T20 Fee                                176 non-null    object
 6   Base_sal_pounds                        176 non-null    object
dtypes: object(7)
memory usage: 9.8+ KB


In [151]:
IntCrickSals.head(30)


,Players,Country,Retainer Fee / contract / base salary,Test Fee,ODI Fee,T20 Fee,Base_sal_pounds
0,Babar Azam,Pakistan,1.25 million PKR (+ undisclosed Captain allowa...,"PKR 50,000","PKR 25,000","PKR 15,000",1.25 million PKR (+ undisclosed Captain allowa...
1,Mohammad Rizwan,Pakistan,1.25 million PKR per month,"PKR 50,000","PKR 25,000","PKR 15,000",1.25 million PKR per month
2,Hasan Ali,Pakistan,1.25 million PKR per month,"PKR 50,000","PKR 25,000","PKR 15,000",1.25 million PKR per month
3,Shaheen Afridi,Pakistan,1.25 million PKR per month,"PKR 50,000","PKR 25,000","PKR 15,000",1.25 million PKR per month
4,Azhar Ali,Pakistan,"937,500 PKRper month","PKR 40,000",0,0,"937,500 PKRper month"
5,Faheem Ashraf,Pakistan,"937,500 PKR per month","PKR 40,000","PKR 20,000","PKR 12,500","937,500 PKR per month"
6,Fakhar Zaman,Pakistan,"937,500 PKR per month","PKR 40,000","PKR 20,000","PKR 12,500","937,500 PKR per month"
7,Fawad Alam,Pakistan,"937,500 PKR per month","PKR 40,000",0,0,"937,500 PKR per month"
8,Shadab Khan,Pakistan,"937,500 PKR per month","PKR 40,000","PKR 20,000","PKR 12,500","937,500 PKR per month"
9,Yasir Shah,Pakistan,"937,500 PKR per month","PKR 40,000","PKR 20,000",0,"937,500 PKR per month"


In [152]:
IntCrickSals['Base_sal_pounds'] = IntCrickSals['Base_sal_pounds'].str.replace(r'\D+', '')
IntCrickSals.head(30)

C:\Users\Pearc\AppData\Local\Temp\ipykernel_35260\3577676305.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  IntCrickSals['Base_sal_pounds'] = IntCrickSals['Base_sal_pounds'].str.replace(r'\D+', '')


,Players,Country,Retainer Fee / contract / base salary,Test Fee,ODI Fee,T20 Fee,Base_sal_pounds
0,Babar Azam,Pakistan,1.25 million PKR (+ undisclosed Captain allowa...,"PKR 50,000","PKR 25,000","PKR 15,000",125
1,Mohammad Rizwan,Pakistan,1.25 million PKR per month,"PKR 50,000","PKR 25,000","PKR 15,000",125
2,Hasan Ali,Pakistan,1.25 million PKR per month,"PKR 50,000","PKR 25,000","PKR 15,000",125
3,Shaheen Afridi,Pakistan,1.25 million PKR per month,"PKR 50,000","PKR 25,000","PKR 15,000",125
4,Azhar Ali,Pakistan,"937,500 PKRper month","PKR 40,000",0,0,937500
5,Faheem Ashraf,Pakistan,"937,500 PKR per month","PKR 40,000","PKR 20,000","PKR 12,500",937500
6,Fakhar Zaman,Pakistan,"937,500 PKR per month","PKR 40,000","PKR 20,000","PKR 12,500",937500
7,Fawad Alam,Pakistan,"937,500 PKR per month","PKR 40,000",0,0,937500
8,Shadab Khan,Pakistan,"937,500 PKR per month","PKR 40,000","PKR 20,000","PKR 12,500",937500
9,Yasir Shah,Pakistan,"937,500 PKR per month","PKR 40,000","PKR 20,000",0,937500


In [153]:
null_series7 = pd.isnull(IntCrickSals['Base_sal_pounds'])
IntCrickSals[null_series7]

,Players,Country,Retainer Fee / contract / base salary,Test Fee,ODI Fee,T20 Fee,Base_sal_pounds
47,Eoin Morgan,England,"£925,000 (plus undisclosed captain allowance)",14500,4500,4500,NaN
48,James Anderson,England,925000,14500,4500,4500,NaN
49,Stuart Broad,England,925000,14500,4500,4500,NaN
50,Joe Root,England,"£925,000 (Undisclosed test captain allowance)",14500,4500,4500,NaN
51,Ben Stokes,England,925000,14500,4500,4500,NaN
52,Jonathan Bairstow,England,925000,14500,4500,4500,NaN
53,Jos Buttler,England,925000,14500,4500,4500,NaN
54,Chris Woakes,England,925000,14500,4500,4500,NaN
55,Moeen Ali,England,925000,14500,4500,4500,NaN
56,Sam Curran,England,925000,14500,4500,4500,NaN


In [154]:
IntCrickSals.loc[47:66, 'Base_sal_pounds'] = 925000
IntCrickSals['Base_sal_pounds'] = IntCrickSals['Base_sal_pounds'].fillna(0)
print(IntCrickSals[null_series7])

                   Players      Country  \
47             Eoin Morgan      England   
48          James Anderson      England   
49            Stuart Broad      England   
50                Joe Root      England   
51              Ben Stokes      England   
52       Jonathan Bairstow      England   
53             Jos Buttler      England   
54            Chris Woakes      England   
55               Moeen Ali      England   
56              Sam Curran      England   
57            Jofra Archer      England   
58               Jason Roy      England   
59               Mark Wood      England   
60             Adil Rashid      England   
61              Rory Burns      England   
62             Zak Crawley      England   
63              Jack Leach      England   
64             Dawid Malan      England   
65              Ollie Pope      England   
66          Ollie Robinson      England   
115       Lachlan Ferguson  New Zealand   
156          Soumya Sarkar   Bangladesh   
158        

In [155]:
IntCrickSals['Base_sal_pounds'] = IntCrickSals['Base_sal_pounds'].astype(int)

In [156]:
IntCrickSals.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 176 entries, 0 to 175
Data columns (total 7 columns):
 #   Column                                 Non-Null Count  Dtype 
---  ------                                 --------------  ----- 
 0   Players                                176 non-null    object
 1   Country                                176 non-null    object
 2   Retainer Fee / contract / base salary  176 non-null    object
 3   Test Fee                               176 non-null    object
 4   ODI Fee                                176 non-null    object
 5   T20 Fee                                176 non-null    object
 6   Base_sal_pounds                        176 non-null    int32 
dtypes: int32(1), object(6)
memory usage: 9.1+ KB


In [157]:
print(IntCrickSals[IntCrickSals["Country"] == "Australia"])

               Players    Country Retainer Fee / contract / base salary  \
67         Aaron Finch  Australia                          $1.5 million   
68           Tim Paine  Australia                            $1 million   
69         Steve Smith  Australia                            $1 million   
70         Pat Cummins  Australia                              $850,000   
71       Glenn Maxwell  Australia                              $850,000   
72        David Warner  Australia                            $1 million   
73      Mitchell Starc  Australia                              $750,000   
74          Adam Zampa  Australia                              $700,000   
75     James Pattinson  Australia                              $500,000   
76      Josh Hazlewood  Australia                              $500,000   
77         Nathan Lyon  Australia                              $500,000   
78         Ashton Agar  Australia                              $500,000   
79          Alex Carey  A

In [158]:
#NOTE - got stuck here trying to remove text from some of numbers cols in order to 
#convert everything into pounds - will leave for now to research and concentrate on joining 
#some of previous
#Eventually sorted out,but laborious
IntCrickSals.loc[0:3, 'Base_sal_pounds'] = 59184
IntCrickSals.loc[4:19, 'Base_sal_pounds'] = IntCrickSals.loc[4:19, 'Base_sal_pounds']*12*0.0035
IntCrickSals.loc[20:46, 'Base_sal_pounds'] = IntCrickSals.loc[20:46, 'Base_sal_pounds']*0.0099
IntCrickSals.loc[155:175, 'Base_sal_pounds'] = IntCrickSals.loc[155:175, 'Base_sal_pounds']*0.81

IntCrickSals.loc[84:99, 'Base_sal_pounds'] = IntCrickSals.loc[84:99, 'Base_sal_pounds']*0.81

IntCrickSals.loc[119:136, 'Base_sal_pounds'] = IntCrickSals.loc[119:136, 'Base_sal_pounds']*0.81

IntCrickSals.loc[137:154, 'Base_sal_pounds'] = IntCrickSals.loc[137:154, 'Base_sal_pounds']*0.81

IntCrickSals.loc[100:118, 'Base_sal_pounds'] = IntCrickSals.loc[100:118, 'Base_sal_pounds']*0.81

IntCrickSals.loc[68:69, 'Base_sal_pounds'] = 807025
IntCrickSals.loc[67, 'Base_sal_pounds'] = 1210537
IntCrickSals.loc[72, 'Base_sal_pounds'] = 59184
IntCrickSals.loc[70:71, 'Base_sal_pounds'] = IntCrickSals.loc[70:71, 'Base_sal_pounds']*0.81
IntCrickSals.loc[73:83, 'Base_sal_pounds'] = IntCrickSals.loc[73:83, 'Base_sal_pounds']*0.81

IntCrickSals.head()
        

,Players,Country,Retainer Fee / contract / base salary,Test Fee,ODI Fee,T20 Fee,Base_sal_pounds
0,Babar Azam,Pakistan,1.25 million PKR (+ undisclosed Captain allowa...,"PKR 50,000","PKR 25,000","PKR 15,000",59184.0
1,Mohammad Rizwan,Pakistan,1.25 million PKR per month,"PKR 50,000","PKR 25,000","PKR 15,000",59184.0
2,Hasan Ali,Pakistan,1.25 million PKR per month,"PKR 50,000","PKR 25,000","PKR 15,000",59184.0
3,Shaheen Afridi,Pakistan,1.25 million PKR per month,"PKR 50,000","PKR 25,000","PKR 15,000",59184.0
4,Azhar Ali,Pakistan,"937,500 PKRper month","PKR 40,000",0,0,39375.0


In [159]:
SoldUnSold21_22.rename(columns = {'PLAYER':'Player'}, inplace = True)
SoldUnSold21_22.head()

,Player,Country,Team,Type,Auction_Base_price,Auc_Base_price_pounds
0,Deepak Chahar,India,Chennai Super Kings,Bowler,140000000,1396080.0
1,Ambati Rayudu,India,Chennai Super Kings,Wicket-Keeper,67500000,673110.0
2,Dwayne Bravo,West Indies,Chennai Super Kings,All-Rounder,44000000,438768.0
3,Shivam Dube,India,Chennai Super Kings,All-Rounder,40000000,398880.0
4,Chris Jordan,England,Chennai Super Kings,All-Rounder,36000000,358992.0


In [160]:
IPLPriceNPerf21_22= SoldUnSold21_22.merge(PerfStats21_22, on='Player', how='left')
IPLPriceNPerf21_22.info()
IPLPriceNPerf21_22.head()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 782 entries, 0 to 781
Data columns (total 31 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Player                 782 non-null    object 
 1   Country                782 non-null    object 
 2   Team                   782 non-null    object 
 3   Type                   782 non-null    object 
 4   Auction_Base_price     782 non-null    int32  
 5   Auc_Base_price_pounds  782 non-null    float64
 6   Bat_Inns_21_22         175 non-null    float64
 7   NOs_21_22              175 non-null    float64
 8   Run_21_22              175 non-null    float64
 9   HS_21                  175 non-null    float64
 10  HS_22                  175 non-null    float64
 11  Bat_Avg_21_22          175 non-null    float64
 12  BF_21_22               175 non-null    float64
 13  BatSR_21_22            175 non-null    float64
 14  100s_21_22             175 non-null    float64
 15  50s_21

,Player,Country,Team,Type,Auction_Base_price,Auc_Base_price_pounds,Bat_Inns_21_22,NOs_21_22,Run_21_22,HS_21,...,Wkts_21_22,Bowl_Avg_21_22,Econ_21_22,BowlSR_21_22,4w_21_22,5w_21_22,Dots_21_22,Catches_21_22,Run outs,Stumpings_21_22
0,Deepak Chahar,India,Chennai Super Kings,Bowler,140000000,1396080.0,4.0,2.0,1.0,1.0,...,14.0,32.214286,8.351852,23.142857,2.0,0.0,132.0,5.0,0.0,0.0
1,Ambati Rayudu,India,Chennai Super Kings,Wicket-Keeper,67500000,673110.0,26.0,4.0,531.0,72.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,6.0,0.0,0.0
2,Dwayne Bravo,West Indies,Chennai Super Kings,All-Rounder,44000000,438768.0,6.0,6.0,70.0,23.0,...,30.0,18.733333,8.363095,13.440000,0.0,0.0,143.0,5.0,0.0,0.0
3,Shivam Dube,India,Chennai Super Kings,All-Rounder,40000000,398880.0,18.0,2.0,519.0,64.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,8.0,8.0,0.0,0.0
4,Chris Jordan,England,Chennai Super Kings,All-Rounder,36000000,358992.0,4.0,1.0,43.0,30.0,...,6.0,38.500000,9.428571,24.500000,0.0,0.0,23.0,4.0,1.5,0.0


Of the 252 players we had performance stats on for 2021 and 2022 in the IPL, we've been able to match 175 who were in the related auctions. The missing c80 are presumably now not playing or weren't bought.

In previous version of the notebook, this was a much lower figure, so it's been worthwhile collecting some extra data. We still have c80 players from the Dec22 auction for the current 2023 season, so we could incorporate those again and try another merge/join, which would pick up a few more, and increase the size of the dataset. But for now, we'll leave this, as it may be useful having the new purchases for 2023 as a separate dataset.

We could also consider ways to search and include players who may not be playing this year, but could be in future years. Perhaps this could be supplied by the full international T20 data.
Let's bring that in now, so we have it to hand


In [161]:
IPLPriceNPerf21_22[IPLPriceNPerf21_22['Player']=='Matthew Wade'].T

,44
Player,Matthew Wade
Country,Australia
Team,Gujarat Titans
Type,Wicket-Keeper
Auction_Base_price,24000000
Auc_Base_price_pounds,239328.0
Bat_Inns_21_22,0.0
NOs_21_22,0.0
Run_21_22,157.0
HS_21,0.0


In [162]:
IntT20Bat = pd.read_csv(r'C:\Users\Pearc\OneDrive\Documents\Data Science\Springboard\SpringboardCapstone2\data\raw_data\International Mens Cricket Stats\Batting\t20.csv')
IntT20Bowl = pd.read_csv(r'C:\Users\Pearc\OneDrive\Documents\Data Science\Springboard\SpringboardCapstone2\data\raw_data\International Mens Cricket Stats\Bowling\Bowling_t20.csv')
IntT20Field = pd.read_csv(r'C:\Users\Pearc\OneDrive\Documents\Data Science\Springboard\SpringboardCapstone2\data\raw_data\International Mens Cricket Stats\Fielding\Fielding_t20.csv')


In [163]:
IntT20Bat.info()
IntT20Bat.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2006 entries, 0 to 2005
Data columns (total 17 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   2006 non-null   int64  
 1   Player       2006 non-null   object 
 2   Span         2006 non-null   object 
 3   Mat          2006 non-null   int64  
 4   Inns         2006 non-null   object 
 5   NO           2006 non-null   object 
 6   Runs         2006 non-null   object 
 7   HS           2006 non-null   object 
 8   Ave          2006 non-null   object 
 9   BF           2006 non-null   object 
 10  SR           2006 non-null   object 
 11  100          2006 non-null   object 
 12  50           2006 non-null   object 
 13  0            2006 non-null   object 
 14  4s           2006 non-null   object 
 15  6s           2006 non-null   object 
 16  Unnamed: 15  0 non-null      float64
dtypes: float64(1), int64(2), object(14)
memory usage: 266.5+ KB


,Unnamed: 0,Player,Span,Mat,Inns,NO,Runs,HS,Ave,BF,SR,100,50,0,4s,6s,Unnamed: 15
0,0,V Kohli (INDIA),2010-2019,75,70,20,2633,94*,52.66,1907,138.07,0,24,2,247,71,NaN
1,1,RG Sharma (INDIA),2007-2019,104,96,14,2633,118,32.1,1905,138.21,4,19,6,234,120,NaN
2,2,MJ Guptill (NZ),2009-2019,83,80,7,2436,105,33.36,1810,134.58,2,15,2,215,113,NaN
3,3,Shoaib Malik (ICC/PAK),2006-2019,111,104,30,2263,75,30.58,1824,124.06,0,7,1,186,61,NaN
4,4,BB McCullum (NZ),2005-2015,71,70,10,2140,123,35.66,1571,136.21,2,13,3,199,91,NaN


In [164]:
IntT20Bowl.info()
IntT20Bowl.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2006 entries, 0 to 2005
Data columns (total 16 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   2006 non-null   int64  
 1   Player       2006 non-null   object 
 2   Span         2006 non-null   object 
 3   Mat          2006 non-null   int64  
 4   Inns         2006 non-null   object 
 5   Overs        2006 non-null   object 
 6   Mdns         2006 non-null   object 
 7   Runs         2006 non-null   object 
 8   Wkts         2006 non-null   object 
 9   BBI          2006 non-null   object 
 10  Ave          2006 non-null   object 
 11  Econ         2006 non-null   object 
 12  SR           2006 non-null   object 
 13  4            2006 non-null   object 
 14  5            2006 non-null   object 
 15  Unnamed: 14  0 non-null      float64
dtypes: float64(1), int64(2), object(13)
memory usage: 250.9+ KB


,Unnamed: 0,Player,Span,Mat,Inns,Overs,Mdns,Runs,Wkts,BBI,Ave,Econ,SR,4,5,Unnamed: 14
0,0,SL Malinga (SL),2006-2019,79,79,284.5,1,2061,106,5/6,19.44,7.23,16.1,1,2,NaN
1,1,Shahid Afridi (ICC/PAK),2006-2018,99,97,361.2,4,2396,98,4/11,24.44,6.63,22.1,3,0,NaN
2,2,Shakib Al Hasan (BDESH),2006-2019,76,75,277.5,2,1894,92,5/20,20.58,6.81,18.1,3,1,NaN
3,3,Saeed Ajmal (PAK),2009-2015,64,63,238.2,2,1516,85,4/19,17.83,6.36,16.8,4,0,NaN
4,4,Umar Gul (PAK),2007-2016,60,60,200.3,2,1443,85,5/6,16.97,7.19,14.1,4,2,NaN


In [165]:
IntT20Field.info()
IntT20Field.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2006 entries, 0 to 2005
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   2006 non-null   int64  
 1   Player       2006 non-null   object 
 2   Span         2006 non-null   object 
 3   Mat          2006 non-null   int64  
 4   Inns         2006 non-null   object 
 5   Dis          2006 non-null   object 
 6   Ct           2006 non-null   object 
 7   St           2006 non-null   object 
 8   Ct Wk        2006 non-null   object 
 9   Ct Fi        2006 non-null   object 
 10  MD           2006 non-null   object 
 11  D/I          2006 non-null   object 
 12  Unnamed: 11  0 non-null      float64
dtypes: float64(1), int64(2), object(10)
memory usage: 203.9+ KB


,Unnamed: 0,Player,Span,Mat,Inns,Dis,Ct,St,Ct Wk,Ct Fi,MD,D/I,Unnamed: 11
0,0,MS Dhoni (INDIA),2006-2019,98,97,91,57,34,57,0,5 (5ct 0st),0.938,NaN
1,1,AB de Villiers (SA),2006-2017,78,78,72,65,7,21,44,4 (4ct 0st),0.9229999999999999,NaN
2,2,D Ramdin (WI),2006-2019,71,71,63,43,20,43,0,4 (4ct 0st),0.887,NaN
3,3,Kamran Akmal (PAK),2006-2017,58,58,60,28,32,28,0,4 (0ct 4st),1.034,NaN
4,4,Mushfiqur Rahim (BDESH),2006-2019,84,82,59,31,28,30,1,3 (1ct 2st),0.7190000000000001,NaN


If need to use this data, will need to work on naming of players col as this won't automatically match with IPL listings

In [166]:
SoldUnSold21_22.to_csv(r'C:\Users\Pearc\OneDrive\Documents\Data Science\Springboard\SpringboardCapstone2\data\interim_data\March23\SoldUnSold21_22cleaneddata.csv')
PerfStats21.to_csv(r'C:\Users\Pearc\OneDrive\Documents\Data Science\Springboard\SpringboardCapstone2\data\interim_data\March23\PerfStats21cleaneddata.csv')
PerfStats22.to_csv(r'C:\Users\Pearc\OneDrive\Documents\Data Science\Springboard\SpringboardCapstone2\data\interim_data\March23\PerfStats22cleaneddata.csv')
PerfStats21_22.to_csv(r'C:\Users\Pearc\OneDrive\Documents\Data Science\Springboard\SpringboardCapstone2\data\interim_data\March23\PerfStats21_22cleaneddata.csv')
IntCrickSals.to_csv(r'C:\Users\Pearc\OneDrive\Documents\Data Science\Springboard\SpringboardCapstone2\data\interim_data\March23\InternatCrickSalspartcleaneddata.csv')
IPLPriceNPerf21_22.to_csv(r'C:\Users\Pearc\OneDrive\Documents\Data Science\Springboard\SpringboardCapstone2\data\interim_data\March23\IPLPriceNPerf21_22cleaneddata.csv')
